In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
# import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
import math
import time
# from sympy import *
from scipy.linalg import schur, eigvals
extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

In [2]:
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of `x` and `y`

    Parameters
    ----------
    x, y : array_like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    Returns
    -------
    matplotlib.patches.Ellipse

    Other parameters
    ----------------
    kwargs : `~matplotlib.patches.Patch` properties
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    print(np.shape(cov))
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0),
        width=ell_radius_x * 2,
        height=ell_radius_y * 2,
        facecolor=facecolor,
        **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)


In [273]:
def generate_localstatsmat_eig(Nparams,Grand):
    '''
    Input 
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    X=np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)

    X[:NE,:NE],X[NE:,NE:]=xee*X[:NE,:NE]/np.sqrt(NE),xii*X[NE:,NE:]/np.sqrt(NI)
    X[:NE,NE:],X[NE:,:NE]=xei*X[:NE,NE:]/np.sqrt(NE),xie*X[NE:,:NE]/np.sqrt(NI)
    # eigvx,eigvecx=la.eig(X)
    # theoretical M (grandom) matrix 
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    # for i in range(2):
    #     gmat[:,i]*=at[i]
    eigvgm,eigvecgm=la.eig(gmat)
    # eigvgm_,eigvecgmleft=la.eig(gmat.T)
    # eigvgm,eigvecgm=schur(gmat) # schur decomposition
    '''
    ugm,svgm,vhgm=la.svd(gmat)
    vgm=vhgm.T
    ngm=vhgm.copy()
    ngm=ngm.T 
    ngm[:,0],ngm[:,1]=svgm[0]*ngm[:,0],svgm[1]*ngm[:,1]
    '''

    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii

    return (X,Gamp,eigvgm,eigvecgm,gaverage)#,ugm,svgm,vgm)

def generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII):
    # mean value 
    # first use rank-1 structure
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI 
    Am=np.zeros((N,N))
    Am[:NE,:NE],Am[:NE,NE:]=JEE/NE,-JEI/NI
    Am[NE:,:NE],Am[NE:,NE:]=JIE/NE,-JII/NI
    ua,svam,vha=la.svd(Am)
    va=vha.T
    # svam=svam[0]
    # ua,va=np.zeros((N,N)),np.zeros((N,N))
    # ua[:,0]=1.0/np.sqrt(N)
    # va[:NE,0]=JE/np.sqrt(NE)
    # va[NE:,0]=-JI/np.sqrt(NI)
    # va[:,0]/=-np.linalg.norm(va[:,0])
    # svam=np.sqrt(2*(JE**2+JI**2))
    # vha=va.T

    Jsv=np.zeros((2,2))
    Jsv[0,0],Jsv[0,1],Jsv[1,0],Jsv[1,1]=JEE,JEI,JIE,JII

    return (Am,Jsv,ua,svam,va)
# generate gmat, same gavg
def gmatamplitude_eig(gavgfix,typenum):
  Amplit = gavgfix*typenum
  numsample = typenum**2
  Amplitg= np.zeros(numsample)
  idxc=0
  while (1):
    if idxc>=numsample:
      Amplitg[numsample-1]=1.0-np.sum(Amplitg[:numsample-1])
      break
    p=np.random.random(1)
    Amplitg[idxc]=np.minimum(p,1.0-np.sum(Amplitg))
    if np.sum(Amplitg)>1.0:
      continue 
    elif np.sum(Amplitg)==1.0:
      break
    else:
      idxc +=1
      # Amplitg[idxc]=np.min(p,1.0-np.sum(Amplitg))
  # Amplitg=0
  Amplitg*=Amplit
  Amplitg=np.sqrt(Amplitg)
  return Amplitg

# Listing all statistical properties
* ## rank-2 eigen spectrum, SVD
* ## rank-1 left $\mathbf{n}$ and right $\mathbf{m}$

In [3]:
''' RANK-1 '''
''' heterogeneous case with single zero g_{EE} '''

# # # 18,10 old will not use again
# strfile = '2021-01-19-11_23_52_heterogeneousNotgEE_cloudsdata'
# strfile ='2021-01-19-12_02_53_heterogeneousgII_cloudsdata'
# strfile='2021-01-19-11_44_14_homogeneous_cloudsdata'
# strfile='2021-01-20-23_54_43_heterogeneousgNotgEE_cloudsdata'


# # 10, 6
# strfile='2021-01-31-22_23_58_homogeneous_cloudsdata'
# strfile = '2021-01-26-13_59_47_heterogeneousNotgEE_cloudsdata'
# strfile ='2021-01-27-13_35_25_heterogeneousrII_cloudsdata'
# strfile='2021-01-27-13_11_16_homogeneous_cloudsdata'

## newest
strfile='2021-02-01-10_59_18_homogeneous_cloudsdata'
strfile='2021-02-01-14_21_42_heterogeneousNotrEE_cloudsdata'
strfile='2021-02-01-15_15_48_homogeneous_cloudsdata'
data = np.load(strfile+'.npz')#'2021-01-19-12_02_53_heterogeneousgII_cloudsdata.npz')
meigseries,neigseries=data['meigseries'],data['neigseries']
gaverageseries=data['gaverageseries']
print(gaverageseries)
# # loading calculated mean and std (2*2) for rank1, 0,0 m,E 0,1 m,I
# meanloadingseries,stdloadingseries=data['meanloadingseries'],data['stdloadingseries']
# ((niter,ntrials,NE*2))
ngavg,nbatch,_=np.shape(meigseries)

# Nt=np.array([1000,1000])
# NE,NI=Nt[0],Nt[1]
# Nparams=np.array([NE,NI])
# JE,JI,a,b=1.0,3.5,-0.6,0.6
# a,b=0.0,0.0
# JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
# Jparameters = np.zeros(4+1)
# Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]=JEE,JEI,JIE,JII
# Jparameters[4]=NE
nrank=1
Jparameters=data['Jparameters']
NE = int(Jparameters[4])
print('Jparams:',Jparameters)

P = np.zeros((2,2))
P = np.array([[Jparameters[0],-Jparameters[1]],[Jparameters[2],-Jparameters[3]]])
Am=np.zeros((NE*2,NE*2))
Am[:NE,:NE],Am[:NE,NE:]=P[0,0]/NE,P[0,1]/NE
Am[NE:,:NE],Am[NE:,NE:]=P[1,0]/NE,P[1,1]/NE
# # calculate the original \bar{m} and \bar{n}
# uAm,svAm,vhAm=la.svd(Am)
# vAm=vhAm.T
# mAm,nAm=uAm[:,:nrank],vAm[:,:nrank]
# for i in range(nrank):
#     nAm[:,i]*=svAm[i]
# print('\bar{m}&\bar{n}')
# print(mAm[1::NE,0],nAm[1::NE,0])
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
print('\bar{m}&\bar{n}')
print(meigvecAm[1::NE,0],neigvecAm[1::NE,0])
print('eigenvalues:',eigvAm[:nrank])
# [-0.02236068 -0.02236068] [-0.04472136  0.15652476] #strfile='2021-01-19-11_44_14_homogeneous_cloudsdata'
# print(np.sum(neigvecAm[:,0]*meigvecAm[:,0]))


[0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65 0.7
 0.75 0.8  0.85 0.9  0.95 1.  ]
Jparams: [   1.     3.5    1.     3.5 1000. ]
ar{m}ar{n}
[0.02236068+0.j 0.02236068+0.j] [ 0.04472136-0.j -0.15652476+0.j]
eigenvalues: [-2.5+0.j]


In [977]:
# # # 18,10 old will not use again
strfile = '2021-01-19-11_23_52_heterogeneousNotgEE_cloudsdata'
# strfile ='2021-01-19-12_02_53_heterogeneousgII_cloudsdata'
# strfile='2021-01-19-11_44_14_homogeneous_cloudsdata'
# strfile='2021-01-20-23_54_43_heterogeneousgNotgEE_cloudsdata'
# strfile='2021-01-31-22_23_58_heterogeneousrII_cloudsdata'
# strfile = '2021-01-26-13_59_47_heterogeneousNotgEE_cloudsdata'
# strfile ='2021-01-27-13_35_25_heterogeneousrII_cloudsdata'
# strfile='2021-01-27-13_11_16_homogeneous_cloudsdata'
data = np.load(strfile+'.npz')#'2021-01-19-12_02_53_heterogeneousgII_cloudsdata.npz')
meigseries,neigseries=data['meigseries'],data['neigseries']
gaverageseries=data['gaverageseries']
print(np.mean(meigseries[18,8,:NE]),np.mean(meigseries[18,8,NE:]))
print(np.std(meigseries[18,8,:NE]),np.std(meigseries[18,8,NE:]))
print(la.norm(meigvecloading[18,8,:]))

# strfile='2021-01-19-11_44_14_homogeneous_cloudsdata'
# strfile='2021-01-20-23_54_43_heterogeneousgNotgEE_cloudsdata'
# strfile='2021-01-31-22_23_58_heterogeneousrII_cloudsdata'
strfile = '2021-01-26-13_59_47_heterogeneousNotgEE_cloudsdata'
# strfile ='2021-01-27-13_35_25_heterogeneousrII_cloudsdata'
# strfile='2021-01-27-13_11_16_homogeneous_cloudsdata'
data = np.load(strfile+'.npz')#'2021-01-19-12_02_53_heterogeneousgII_cloudsdata.npz')
meigseries,neigseries=data['meigseries'],data['neigseries']
gaverageseries=data['gaverageseries']
print(np.mean(meigseries[18,8,:NE]),np.mean(meigseries[18,8,NE:]))
print(np.std(meigseries[18,8,:NE]),np.std(meigseries[18,8,NE:]))
print(la.norm(meigvecloading[18,8,:]))

strfile = '2021-01-31-23_52_01_homogeneousRL_cloudsdata'
# strfile ='2021-01-27-13_35_25_heterogeneousrII_cloudsdata'
# strfile='2021-01-27-13_11_16_homogeneous_cloudsdata'
data = np.load(strfile+'.npz')#'2021-01-19-12_02_53_heterogeneousgII_cloudsdata.npz')
meigseries,neigseries=data['meigseries'],data['neigseries']
gaverageseries=data['gaverageseries']
print(np.mean(meigseries[18,8,:NE]),np.mean(meigseries[18,8,NE:]))
print(np.std(meigseries[18,8,:NE]),np.std(meigseries[18,8,NE:]))
print(la.norm(meigvecloading[18,8,:]))

0.020544469970274783 0.02098986466579113
0.006655963985015973 0.00964616392056558
0.9978334240927988
-0.02161063400644167 -0.022155969669385343
0.00442547307356689 0.004744332821803143
0.9978334240927988
0.022027923451546725 0.0219495616429156
0.004101373666487598 0.004020704682575918
0.9978334240927988


In [944]:
# ngavg,nbatch,_=np.shape(meigseries)
print(la.norm(meigseries[10,8,:NE]),la.norm(neigseries[10,8,:NE]))

0.7071387554442274 1.6425955406819934


## eigenvalues in RANK-1

In [9]:
# generate figure
xticks = np.linspace(0.0,1.0,2)
xlims = [-0.1,1.1]
ytick2 = np.linspace(-1.50,-1.20,2)
ylim2 = [-1.51,-1.19]
ytick1 = np.linspace(-3.0,-2.0,2)
ylim1 = [-3.01,-1.99]
theta = np.linspace(0, 2 * np.pi, 200)
# fig,ax=plt.subplots(1,3,figsize=(12,4))
# ''' PLOT a sample scatter plot A '''
# ## three cases
# case one homogeneous
strfile='2021-01-27-13_11_16_homogeneous_cloudsdata'
data = np.load(strfile+'.npz')
gaverageseries=data['gaverageseries']

nrank=1
Jparameters=data['Jparameters']
JEE,JIE,JEI,JII=Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]
Nparams=np.array([NE,NE])
# generate mean 
Am,Jsv,ua,svam,va=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)

# OVERALL
eigvalAm,_=la.eig(Am)
# eigenvalues
eigvalseries=data['eigvalseries']
# plot the single outliers   
fig,ax=plt.subplots(1,3,figsize=(9,1.5))
idxxx=0
for i in range(20):
    ax[idxxx].scatter(gaverageseries[i]*np.ones(20),eigvalseries[i,:,0],s=5.0, c='red', alpha=0.5)
# numerical mean and std
meanlambda,stdlambda=np.zeros((ngavg,nrank)),np.zeros((ngavg,nrank))
for irank in range(nrank):
    meanlambda[:,irank]=np.mean(eigvalseries[:,:,irank],axis=1)
    stdlambda[:,irank]=np.std(eigvalseries[:,:,irank],axis=1)
ax[idxxx].plot(gaverageseries,meanlambda[:,0], lw=2, label=r'$\lambda_1$', color='red')
ax[idxxx].fill_between(gaverageseries,meanlambda[:,0]+stdlambda[:,0], meanlambda[:,0]-stdlambda[:,0], facecolor='orange', alpha=0.5)
ax[idxxx].plot(gaverageseries,eigvalAm[0]*np.ones(ngavg),'k--',linewidth=1.5,label=r'$\lambda_1^O$')

ax[idxxx].set_xlim(xlims)
ax[idxxx].set_xticks([])#xticks)#,fontsize=12)
    # ax[irank][idxxx].set_aspect('equal')
ax[idxxx].set_xticks(xticks)
ax[idxxx].set_ylim(ylim1)
ax[idxxx].set_yticks(ytick1)#,fontsize=12)
plt.rc('xtick',labelsize=9)
plt.rc('ytick',labelsize=9)
# ax.legend()#["mean outlier","original outlier"],fontsize=14)
    
''' case one hETEROGENEOUS RII '''
strfile ='2021-01-27-13_35_25_heterogeneousrII_cloudsdata'
data = np.load(strfile+'.npz')
# eigenvalues
eigvalseries=data['eigvalseries']
# plot the single outliers   
# fig,ax=plt.subplots(2,3,figsize=(9,3))
idxxx+=1
for i in range(15):
    ax[idxxx].scatter(gaverageseries[i]*np.ones(20),eigvalseries[i,:,0],s=5.0, c='red', alpha=0.5)
# numerical mean and std
meanlambda,stdlambda=np.zeros((ngavg,nrank)),np.zeros((ngavg,nrank))
for irank in range(nrank):
    meanlambda[:,irank]=np.mean(eigvalseries[:,:,irank],axis=1)
    stdlambda[:,irank]=np.std(eigvalseries[:,:,irank],axis=1)
ax[idxxx].plot(gaverageseries[:15],meanlambda[:15,0], lw=2, label=r'$\lambda_1$', color='red')
ax[idxxx].fill_between(gaverageseries[:15],meanlambda[:15,0]+stdlambda[:15,0], meanlambda[:15,0]-stdlambda[:15,0], facecolor='orange', alpha=0.5)
ax[idxxx].plot(gaverageseries[:],eigvalAm[0]*np.ones(ngavg),'k--',linewidth=1.5,label=r'$\lambda_1^O$')

ax[idxxx].set_xlim(xlims)
ax[idxxx].set_xticks([])#xticks)#,fontsize=12)
    # ax[irank][idxxx].set_aspect('equal')
ax[idxxx].set_xticks(xticks)
ax[idxxx].set_ylim(ylim1)
ax[idxxx].set_yticks([])#ytick1)#,fontsize=12)
plt.rc('xtick',labelsize=9)
plt.rc('ytick',labelsize=9)

''' case two heterogeneous NotrEE '''
strfile = '2021-01-26-13_59_47_heterogeneousNotgEE_cloudsdata'
data = np.load(strfile+'.npz')
# eigenvalues
eigvalseries=data['eigvalseries']
# plot the single outliers   
# fig,ax=plt.subplots(2,3,figsize=(9,3))
idxxx+=1
for i in range(20):
    ax[idxxx].scatter(gaverageseries[i]*np.ones(20),eigvalseries[i,:,0],s=5.0, c='red', alpha=0.5)
# numerical mean and std
meanlambda,stdlambda=np.zeros((ngavg,nrank)),np.zeros((ngavg,nrank))
for irank in range(nrank):
    meanlambda[:,irank]=np.mean(eigvalseries[:,:,irank],axis=1)
    stdlambda[:,irank]=np.std(eigvalseries[:,:,irank],axis=1)
ax[idxxx].plot(gaverageseries,meanlambda[:,0], lw=2, label=r'$\lambda_1$', color='red')
ax[idxxx].fill_between(gaverageseries,meanlambda[:,0]+stdlambda[:,0], meanlambda[:,0]-stdlambda[:,0], facecolor='orange', alpha=0.5)
ax[idxxx].plot(gaverageseries,eigvalAm[0]*np.ones(ngavg),'k--',linewidth=1.5,label=r'$\lambda_1^O$')
ax[idxxx].set_xlim(xlims)
ax[idxxx].set_xticks([])#xticks)#,fontsize=12)
    # ax[irank][idxxx].set_aspect('equal')
ax[idxxx].set_xticks(xticks)
ax[idxxx].set_ylim(ylim1)
ax[idxxx].set_yticks([])#ytick1)#,fontsize=12)
plt.rc('xtick',labelsize=9)
plt.rc('ytick',labelsize=9)

# ax[2].grid()




FileNotFoundError: [Errno 2] No such file or directory: '2021-01-27-13_11_16_homogeneous_cloudsdata.npz'

In [880]:
idxgavg=10
# generate figure
xticks = np.linspace(-4.0,1.0,5)
xlims = [-4.1,1.1]
yticks = np.linspace(-0.8,0.8,3)
ylims = [-0.81,0.81]
theta = np.linspace(0, 2 * np.pi, 200)
# fig,ax=plt.subplots(1,3,figsize=(12,4))
# ''' PLOT a sample scatter plot A '''
# ## three cases
# case one homogeneous
strfile='2021-01-19-11_44_14_homogeneous_cloudsdata'
data = np.load(strfile+'.npz')
gaverageseries=data['gaverageseries']
nrank=1
Jparameters=data['Jparameters']
JEE,JIE,JEI,JII=Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]
Nparams=np.array([NE,NE])
# generate mean 
Am,Jsv,ua,svam,va=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# OVERALL
eigvalAm,_=la.eig(Am)
# generate X
# totally average
gaverage=gaverageseries[idxgavg]
grand = gaverage*np.ones(4)/np.sqrt(2.0)
print('pre avg:',gaverage)
## homogeneous X matrix
X,Gamp,eigvgmT,eigvecgm,gaverage=generate_localstatsmat_eig(Nparams,grand)#,ugm,svgm,vgm
print('post avg:',gaverage)
xr = gaverage*np.cos(theta)
yr = gaverage*np.sin(theta)
# mix together
Xtotal=X.copy()+Am.copy()
eigvxt,eigvecxt=la.eig(Xtotal)
# eigenvalues
meigseries,neigseries=data['meigseries'],data['neigseries']
eigvalseries=np.zeros(20)
for i in range(20):
    eigvalseries[i]=np.squeeze(neigseries[idxgavg,i,:])*np.squeeze(meigseries[idxgavg,i,:])

fig,ax=plt.subplots(3,1,figsize=(8,8))
real_eigsbulk,imag_eigsbulk=np.real(eigvxt[nrank:]),np.imag(eigvxt[nrank:])
ax[0].scatter(real_eigsbulk,imag_eigsbulk,marker='^',s=5,color='blue',alpha=0.25)
ax[0].scatter(np.real(eigvxt[:nrank]),np.imag(eigvxt[:nrank]),marker='^',s=5,color='blue',alpha=0.5)
for i in range(nrank):
    ax[0].scatter(np.real(eigvalseries),np.imag(eigvalseries),marker='^',s=5,color='blue',alpha=0.5)
xa = gaverage*np.cos(theta)
ya = gaverage*np.sin(theta)
ax[0].plot(xa, ya, color="darkred", linewidth=1.5,label=r'$r_g$')
ax[0].plot(xr, yr, color="grey", linewidth=1.5,linestyle='--',label=r'fixed $\bar{g}$')
ax[0].scatter(np.real(eigvalAm[:nrank]),np.imag(eigvalAm[:nrank]),marker='^',s=15,color='red',alpha=1.0)
ax[0].set_xlim(xlims)
ax[0].set_ylim(ylims)
ax[0].set_xticks(xticks)
ax[0].set_yticks(yticks)
ax[0].set_aspect('equal')
ax[0].legend()
ax[0].grid()

# case two heterogeneous RII
strfile ='2021-01-19-12_02_53_heterogeneousgII_cloudsdata'
data = np.load(strfile+'.npz')
gaverageseries=data['gaverageseries']
# generate X
# max r_g
grand = np.zeros(4)
grand[3]=gaverage*np.sqrt(2) # I dominant
print('pre avg:',gaverage)
## homogeneous X matrix
X,Gamp,eigvgmT,eigvecgm,gaverage=generate_localstatsmat_eig(Nparams,grand)#,ugm,svgm,vgm
print('post avg:',gaverage)

# mix together
Xtotal=X.copy()+Am.copy()
eigvxt,eigvecxt=la.eig(Xtotal)
# eigenvalues
meigseries,neigseries=data['meigseries'],data['neigseries']
eigvalseries=np.zeros(20)
for i in range(20):
    eigvalseries[i]=np.squeeze(neigseries[idxgavg,i,:])*np.squeeze(meigseries[idxgavg,i,:])
# np.squeeze(meigseries[i,:,:NE,j]).T
real_eigsbulk,imag_eigsbulk=np.real(eigvxt[nrank:]),np.imag(eigvxt[nrank:])
ax[1].scatter(real_eigsbulk,imag_eigsbulk,marker='^',s=5,color='blue',alpha=0.25)
ax[1].scatter(np.real(eigvxt[:2]),np.imag(eigvxt[:2]),marker='^',s=5,color='blue',alpha=0.5)
for i in range(nrank):
    ax[1].scatter(np.real(eigvalseries),np.imag(eigvalseries),marker='^',s=5,color='blue',alpha=0.5)
xa = np.sqrt(2)*gaverage*np.cos(theta)
ya = np.sqrt(2)*gaverage*np.sin(theta)
ax[1].plot(xa, ya, color="darkred", linewidth=1.5,label=r'$r_g$')
ax[1].plot(xr, yr, color="grey", linewidth=1.5,linestyle='--',label=r'fixed $\bar{g}$')
ax[1].scatter(np.real(eigvalAm[:nrank]),np.imag(eigvalAm[:nrank]),marker='^',s=15,color='red',alpha=1.0)
ax[1].set_xlim(xlims)
ax[1].set_ylim(ylims)
ax[1].set_xticks(xticks)
ax[1].set_yticks(yticks)
ax[1].set_aspect('equal')
ax[0].set_aspect('equal')
ax[1].legend()
ax[1].grid()

# case two heterogeneous not gEE
strfile = '2021-01-19-11_23_52_heterogeneousNotgEE_cloudsdata'
data = np.load(strfile+'.npz')
gaverageseries=data['gaverageseries']
# generate X
# three copy
grand = gaverage*np.ones(4)*np.sqrt(2.0/3.0)
grand[0]=0
print('pre avg:',gaverage)
## homogeneous X matrix
X,Gamp,eigvgmT,eigvecgm,gaverage=generate_localstatsmat_eig(Nparams,grand)#,ugm,svgm,vgm
print('post avg:',gaverage)

# mix together
Xtotal=X.copy()+Am.copy()
eigvxt,eigvecxt=la.eig(Xtotal)
# eigenvalues
# eigenvalues
meigseries,neigseries=data['meigseries'],data['neigseries']
eigvalseries=np.zeros(20)
for i in range(20):
    eigvalseries[i]=np.squeeze(neigseries[idxgavg,i,:])*np.squeeze(meigseries[idxgavg,i,:])

real_eigsbulk,imag_eigsbulk=np.real(eigvxt[2:]),np.imag(eigvxt[2:])
ax[2].scatter(real_eigsbulk,imag_eigsbulk,marker='^',s=5,color='blue',alpha=0.25)
ax[2].scatter(np.real(eigvxt[:nrank]),np.imag(eigvxt[:nrank]),marker='^',s=5,color='blue',alpha=0.5)
for i in range(nrank):
    ax[2].scatter(np.real(eigvalseries),np.imag(eigvalseries),marker='^',s=5,color='blue',alpha=0.5)
arg = np.sqrt((1+np.sqrt(5))/3.0)
xa = arg*gaverage*np.cos(theta)
ya = arg*gaverage*np.sin(theta)
ax[2].plot(xa, ya, color="darkred", linewidth=1.5,label=r'$r_g$')
ax[2].plot(xr, yr, color="grey", linewidth=1.5,linestyle='--',label=r'fixed $\bar{g}$')
ax[2].scatter(np.real(eigvalAm[:nrank]),np.imag(eigvalAm[:nrank]),marker='^',s=15,color='red',alpha=1.0)
ax[2].set_xlim(xlims)
ax[2].set_ylim(ylims)
ax[2].set_xticks(xticks)
ax[2].set_yticks(yticks)
ax[2].set_aspect('equal')
ax[0].set_aspect('equal')
ax[2].legend()
ax[2].grid()




pre avg: 0.5499999999999999
post avg: 0.5499999999999998


ValueError: setting an array element with a sequence.

## [Fig.] choose the data for m and n, delete data that are far away from the standard range

In [873]:
meigvecloading,neigvecloading=data['meigseries'],data['neigseries']
for igavg in range(ngavg):
    meanmE = np.mean(np.squeeze(meigvecloading[igavg,:,:NE]),axis=1)#np.zeros((ngavg,2,nrank*2,nbatch)),
    # print(np.shape(meanmE))
    idxmE =np.where(meanmE<0)
    meigvecloading[igavg,idxmE,:NE]*=-1
    meanmI = np.mean(np.squeeze(meigvecloading[igavg,:,NE:]),axis=1)
    idxmI =np.where(meanmI<0)
    meigvecloading[igavg,idxmI,NE:]*=-1

    meannE = np.mean(np.squeeze(neigvecloading[igavg,:,:NE]),axis=1)#np.zeros((ngavg,2,nrank*2,nbatch)),
    idxnE =np.where(meannE<0)
    neigvecloading[igavg,idxnE,:NE]*=-1
    meannI = np.mean(np.squeeze(neigvecloading[igavg,:,NE:]),axis=1)
    idxnI =np.where(meannI>0)
    neigvecloading[igavg,idxnI,NE:]*=-1
## puring
meanmE,meanmI,meannE,meannI=np.zeros((ngavg,nrank)),np.zeros((ngavg,nrank)),np.zeros((ngavg,nrank)),np.zeros((ngavg,nrank))
for igavg in range(ngavg):
    # puring E
    mEmat = np.squeeze(meigvecloading[igavg,:,:NE])
    meanmEtotal,stdmEtotal = np.mean(np.mean(mEmat)),np.std(np.reshape(mEmat,(NE*nbatch)))
    varmE = mEmat - meanmEtotal
    idxwhere = np.where(np.abs(varmE)>3.0*stdmEtotal)
    mEmat[idxwhere]=meanmEtotal
    meigvecloading[igavg,:,:NE]=mEmat
    meanmE[igavg,0]=np.mean(np.mean(mEmat))

    # puring I
    mImat = np.squeeze(meigvecloading[igavg,:,NE:])
    meanmItotal,stdmItotal = np.mean(np.mean(mImat)),np.std(np.reshape(mImat,(NE*nbatch)))
    varmI = mImat - meanmItotal
    idxwhere = np.where(np.abs(varmI)>3.0*stdmItotal)
    mImat[idxwhere]=meanmItotal
    meigvecloading[igavg,:,NE:]=mImat
    meanmI[igavg,0]=np.mean(np.mean(mImat))
    
    # puring E
    nEmat = np.squeeze(neigvecloading[igavg,:,:NE])
    meannEtotal,stdnEtotal = np.mean(np.mean(nEmat)),np.std(np.reshape(nEmat,(NE*nbatch)))
    varnE = nEmat - meannEtotal
    idxwhere = np.where(np.abs(varnE)>3.0*stdnEtotal)
    nEmat[idxwhere]=meannEtotal
    neigvecloading[igavg,:,:NE]=nEmat
    meannE[igavg,0]=np.mean(np.mean(nEmat))

    # puring I
    nImat = np.squeeze(neigvecloading[igavg,:,NE:])
    meannItotal,stdnItotal = np.mean(np.mean(nImat)),np.std(np.reshape(nImat,(NE*nbatch)))
    varnI = nImat - meannItotal
    idxwhere = np.where(np.abs(varnI)>3.0*stdnItotal)
    nImat[idxwhere]=meannItotal
    neigvecloading[igavg,:,NE:]=nImat
    meannI[igavg,0]=np.mean(np.mean(nImat))

''' calculate directions '''
noiseE,noiseI = np.zeros((ngavg,NE*nbatch,2)),np.zeros((ngavg,NE*nbatch,2))
dirvecE,dirvecI=np.zeros((ngavg,2,2)),np.zeros((ngavg,2,2))
for i in range(ngavg):
    noiseE[i,:,0],noiseI[i,:,0],noiseE[i,:,1],noiseI[i,:,1]=np.squeeze(np.reshape(np.squeeze(meigvecloading[i,:,:NE]),(NE*nbatch,1)))-meanmE[i,0],np.squeeze(np.reshape(np.squeeze(meigvecloading[i,:,NE:]),(NE*nbatch,1)))-meanmI[i,0],np.squeeze(np.reshape(np.squeeze(neigvecloading[i,:,:NE]),(NE*nbatch,1)))-meannE[i,0],np.squeeze(np.reshape(np.squeeze(neigvecloading[i,:,NE:]),(NE*nbatch,1)))-meannI[i,0]
    covdirE,covdirI=np.squeeze(noiseE[i,:,:]).T@np.squeeze(noiseE[i,:,:]),np.squeeze(noiseI[i,:,:]).T@np.squeeze(noiseI[i,:,:])
    _,dirvecE[i,:,:]=la.eig(covdirE)
    _,dirvecI[i,:,:]=la.eig(covdirI)

# 18,10
xlims =[-0.51,0.41]
xticks = np.linspace(-0.5,0.4,3)
ylims =[-0.011,0.051]
yticks = np.linspace(-0.01,0.05,3)

# figure for eigenvectors
fig,ax=plt.subplots(1,3,figsize=(12,4))
idxxx=1
idxgavg,idxtrial=18,10#10,8#
ax[idxxx].scatter(neigvecloading[idxgavg,idxtrial,NE:],meigvecloading[idxgavg,idxtrial,NE:],s=5.0,c='blue',alpha=0.5)
ax[idxxx].scatter(neigvecloading[idxgavg,idxtrial,:NE],meigvecloading[idxgavg,idxtrial,:NE],s=5.0,c='red',alpha=0.5)
ax[idxxx].set_xlim(xlims)
ax[idxxx].set_xticks(xticks)
ax[idxxx].set_ylim(ylims)
ax[idxxx].set_yticks([])#yticks)
ax[idxxx].plot([meannE[idxgavg,0],meannE[idxgavg,0]+dirvecE[idxgavg,0,0]],[meanmE[idxgavg,0],meanmE[idxgavg,0]+dirvecE[idxgavg,1,0]],color='r',linestyle='--',linewidth=1.5)
ax[idxxx].plot([meannE[idxgavg,0],meannE[idxgavg,0]+dirvecE[idxgavg,0,1]],[meanmE[idxgavg,0],meanmE[idxgavg,0]+dirvecE[idxgavg,1,1]],color='r',linestyle='--',linewidth=1.5)
ax[idxxx].plot([meannI[idxgavg,0],meannI[idxgavg,0]+dirvecI[idxgavg,1,0]],[meanmI[idxgavg,0],meanmI[idxgavg,0]+dirvecI[idxgavg,0,0]],color='b',linestyle='--',linewidth=1.5)
ax[idxxx].plot([meannI[idxgavg,0],meannI[idxgavg,0]+dirvecI[idxgavg,1,1]],[meanmI[idxgavg,0],meanmI[idxgavg,0]+dirvecI[idxgavg,0,1]],color='b',linestyle='--',linewidth=1.5)
nE,mE=neigvecloading[idxgavg,idxtrial,:NE].copy(),meigvecloading[idxgavg,idxtrial,:NE].copy()
confidence_ellipse(np.real(nE),np.real(mE),ax[idxxx],edgecolor='darkred')
nI,mI=neigvecloading[idxgavg,idxtrial,NE:].copy(),meigvecloading[idxgavg,idxtrial,NE:].copy()
confidence_ellipse(np.real(nI),np.real(mI),ax[idxxx],edgecolor='green')
ax[idxxx].axvline(c='grey', lw=1)
ax[idxxx].axhline(c='grey', lw=1)
# for i in range(3):
#     ax[i].grid()
plt.show()

(2, 2)
(2, 2)


## overlap

In [874]:
xlims =[-.1,1.1]
xticks = np.linspace(0,1.0,3)
ylims =[-10.1,5.1]
yticks = np.linspace(-10.,5.,3)
ylims1 =[-5.1,5.1]
yticks1 = np.linspace(-5.,5.,3)

overlaptt,overlapmean=np.zeros((ngavg,3)) ,np.zeros((ngavg,3)) # rank2,rank2,EITOTAL
# figure for eigenvectors
fig,ax=plt.subplots(figsize=(4,2))
#m1n1
for igavg in range(ngavg):
    # total
    overlaptt[igavg,0]=np.sum(np.squeeze(neigvecloading[idxgavg,idxtrial,:NE])*np.squeeze(meigvecloading[idxgavg,idxtrial,:NE]))
    overlaptt[igavg,1]=np.sum(np.squeeze(neigvecloading[idxgavg,idxtrial,NE:])*np.squeeze(meigvecloading[idxgavg,idxtrial,NE:]))
    overlaptt[igavg,2]=np.sum(np.squeeze(neigvecloading[idxgavg,idxtrial,:])*np.squeeze(meigvecloading[idxgavg,idxtrial,:]))
    # mean
    overlapmean[igavg,0]=np.squeeze(meannE[idxgavg,0])*np.squeeze(meanmE[idxgavg,0])*NE
    overlapmean[igavg,1]=np.squeeze(meannI[idxgavg,0])*np.squeeze(meanmI[idxgavg,0])*NE
    overlapmean[igavg,2]=np.squeeze(meannE[idxgavg,0])*np.squeeze(meanmE[idxgavg,0])*NE+np.squeeze(meannI[idxgavg,0])*np.squeeze(meanmI[idxgavg,0])*NE

ax.plot(gaverageseries,overlaptt[:,0],color='orange',linestyle='-',linewidth=2.5,alpha=0.5) # final type
ax.plot(gaverageseries,overlapmean[:,0],color='red',linestyle='--',linewidth=1.5,alpha=1.0) # final type
ax.plot(gaverageseries,overlaptt[:,1],color='cyan',linestyle='-',linewidth=2.5,alpha=0.5) # final type
ax.plot(gaverageseries,overlapmean[:,1],color='blue',linestyle='--',linewidth=1.5,alpha=1.0) # final type
ax.plot(gaverageseries,overlaptt[:,2],color='gray',linestyle='-',linewidth=2.5,alpha=0.5) # final type
ax.plot(gaverageseries,overlapmean[:,2],color='black',linestyle='--',linewidth=1.5,alpha=1.0) # final type
# ax[0].set_xlim(xlims)
# ax[0].set_xticks([])#xticks)
# ax[0].set_ylim(ylims)
# ax[0].set_yticks(yticks)
# for i in np.arange(1,4,1):
#     ax[i].set_xlim(xlims)
#     # ax[i].set_xticks(xticks)
#     ax[i].set_ylim(ylims1)
#     ax[i].set_yticks(yticks1)
ax[3].set_xticks(xticks)



TypeError: 'AxesSubplot' object is not subscriptable

In [4]:
''' For rank 1 '''
# calculate \theta overlap
loadingeigvec=np.zeros((ngavg,2,nrank*2,NE,nbatch)) # ng,EI,nrank*2 mn,Npopulation,ntrials
loadingeigvecAm=np.zeros((2,nrank*2,NE))
meanloadingeigclouds,stdloadingeigclouds = np.zeros((ngavg,2,nrank*2,nbatch)),np.zeros((ngavg,2,nrank*2,nbatch))
Norm2=np.zeros((ngavg,3,nrank*2,nbatch))
Norm2Am=np.zeros((3,nrank*2,nbatch))
overlapeig,overlapeig_renorm=np.zeros((ngavg,3,nrank*2,nbatch)),np.zeros((ngavg,3,nrank*2,nbatch))
# lambda reconstruction
lambda_reconstruct=np.zeros((ngavg,3,nrank,nbatch)) # E pop, I pop, overall
for j in range(nrank):
    # eig
    loadingeigvecAm[0,j,:]=np.squeeze(meigvecAm[:NE,j]).T # m_j^E
    loadingeigvecAm[0,j+nrank,:]=np.squeeze(neigvecAm[:NE,j]).T # n_j^E
    loadingeigvecAm[1,j,:]=np.squeeze(meigvecAm[NE:,j]).T # m_j^I
    loadingeigvecAm[1,j+nrank,:]=np.squeeze(neigvecAm[NE:,j]).T # n_j^I
    Norm2Am[0,j]=np.sum(np.squeeze(loadingeigvecAm[0,j,:])**2,axis=0) # excitatory
    Norm2Am[1,j]=np.sum(np.squeeze(loadingeigvecAm[1,j,:])**2,axis=0) # inhibitory
    Norm2Am[2,j]=Norm2Am[1,j]+Norm2Am[0,j] # overall
    Norm2Am[0,j+nrank]=np.sum(np.squeeze(loadingeigvecAm[0,j+nrank,:])**2,axis=0) # excitatory
    Norm2Am[1,j+nrank]=np.sum(np.squeeze(loadingeigvecAm[1,j+nrank,:])**2,axis=0) # inhibitory
    Norm2Am[2,j+nrank]=Norm2Am[1,j+nrank]+Norm2Am[0,j+nrank] # overall
for j in range(nrank):
    for i in range(ngavg):
        # eig
        loadingeigvec[i,0,j,:,:]=np.squeeze(meigseries[i,:,:NE]).T # m_j^E
        loadingeigvec[i,0,j+nrank,:,:]=np.squeeze(neigseries[i,:,:NE]).T # n_j^E
        loadingeigvec[i,1,j,:,:]=np.squeeze(meigseries[i,:,NE:]).T # m_j^I
        loadingeigvec[i,1,j+nrank,:,:]=np.squeeze(neigseries[i,:,NE:]).T # n_j^I

        # renormalization
        # m, right eigenvectors
        Norm2[i,0,j,:]=np.sum(np.squeeze(loadingeigvec[i,0,j,:,:])**2,axis=0) # excitatory
        Norm2[i,1,j,:]=np.sum(np.squeeze(loadingeigvec[i,1,j,:,:])**2,axis=0) # inhibitory
        Norm2[i,2,j,:]=Norm2[i,1,j,:]+Norm2[i,0,j,:] # overall
        # n, left eigenvectors
        Norm2[i,0,j+nrank,:]=np.sum(np.squeeze(loadingeigvec[i,0,j+nrank,:,:])**2,axis=0) # excitatory n
        Norm2[i,1,j+nrank,:]=np.sum(np.squeeze(loadingeigvec[i,1,j+nrank,:,:])**2,axis=0) # inhibitory n
        Norm2[i,2,j+nrank,:]=Norm2[i,1,j+nrank,:]+Norm2[i,0,j+nrank,:] # overall normalization
        # overlap
        # m
        overlapeig[i,0,j,:]=np.squeeze(np.reshape(meigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j,:,:])) # excitatory overlap -- renormalized
        overlapeig[i,1,j,:]=np.squeeze(np.reshape(meigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j,:,:])) # inhibitory overlap -- renromalized
        overlapeig[i,2,j,:]=np.squeeze(np.reshape(meigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j,:,:]))+np.squeeze(np.reshape(meigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j,:,:])) # inhibitory overlap -- renormalized
        # n
        overlapeig[i,0,j+nrank,:]=np.squeeze(np.reshape(neigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j+nrank,:,:]))/(eigvAm[j])**2 # excitatory overlap -- renormalized
        overlapeig[i,1,j+nrank,:]=np.squeeze(np.reshape(neigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j+nrank,:,:]))/(eigvAm[j])**2# inhibitory overlap -- renormalize+
        overlapeig[i,2,j+nrank,:]=np.squeeze(np.reshape(neigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j+nrank,:,:]))/(eigvAm[j])**2+np.squeeze(np.reshape(neigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j+nrank,:,:]))/(eigvAm[j])**2# inhibitory overlap -- renormalized

        # overlap renormalizing
        # m
        overlapeig_renorm[i,0,j,:]=np.squeeze(np.reshape(meigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j,:,:]))/np.sqrt(Norm2[i,0,j,:])/np.sqrt(Norm2Am[0,j]) # excitatory overlap -- renormalized
        overlapeig_renorm[i,1,j,:]=np.squeeze(np.reshape(meigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j,:,:]))/np.sqrt(Norm2[i,1,j,:])/np.sqrt(Norm2Am[1,j]) # inhibitory overlap -- renromalized
        overlapeig_renorm[i,2,j,:]=(np.squeeze(np.reshape(meigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j,:,:]))+np.squeeze(np.reshape(meigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j,:,:])))/np.sqrt(Norm2[i,2,j,:])/np.sqrt(Norm2Am[2,j]) # overlap -- renromalized
        # n
        overlapeig_renorm[i,0,j+nrank,:]=np.squeeze(np.reshape(neigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j+nrank,:,:]))/np.sqrt(Norm2[i,0,j+nrank,:])/np.sqrt(Norm2Am[0,j+nrank]) # excitatory overlap -- renormalized
        overlapeig_renorm[i,1,j+nrank,:]=np.squeeze(np.reshape(neigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j+nrank,:,:]))/np.sqrt(Norm2[i,1,j+nrank,:])/np.sqrt(Norm2Am[1,j+nrank]) # inhibitory overlap -- renormalized
        overlapeig_renorm[i,2,j+nrank,:]=(np.squeeze(np.reshape(neigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j+nrank,:,:]))+np.squeeze(np.reshape(neigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j+nrank,:,:])))/np.sqrt(Norm2[i,2,j+nrank,:])/np.sqrt(Norm2Am[2,j+nrank])
        # statistics
        meanloadingeigclouds[i,0,j,:]=(np.mean(np.squeeze(loadingeigvec[i,0,j,:,:]),axis=0)) # m excitatory trial averaging
        meanloadingeigclouds[i,0,j+nrank,:]=(np.mean(np.squeeze(loadingeigvec[i,0,j+nrank,:,:]),axis=0)) # n excitatory trial average
        meanloadingeigclouds[i,1,j,:]=(np.mean(np.squeeze(loadingeigvec[i,1,j,:,:]),axis=0)) # m inhibitory trial average
        meanloadingeigclouds[i,1,j+nrank,:]=(np.mean(np.squeeze(loadingeigvec[i,1,j+nrank,:,:]),axis=0)) # n inhibitory trial average

        stdloadingeigclouds[i,0,j,:]=np.std(np.squeeze(loadingeigvec[i,0,j,:,:]),axis=0)
        stdloadingeigclouds[i,0,j+nrank,:]=np.std(np.squeeze(loadingeigvec[i,0,j+nrank,:,:]),axis=0)
        stdloadingeigclouds[i,1,j,:]=np.std(np.squeeze(loadingeigvec[i,1,j,:,:]),axis=0)
        stdloadingeigclouds[i,1,j+nrank,:]=np.std(np.squeeze(loadingeigvec[i,1,j+nrank,:,:]),axis=0)

        # reproduce lambda overall
        lambda_reconstruct[i,0,j,:]=np.sum(np.squeeze(meigseries[i,:,:NE])*np.squeeze(neigseries[i,:,:NE]),axis=1)
        lambda_reconstruct[i,1,j,:]=np.sum(np.squeeze(meigseries[i,:,NE:])*np.squeeze(neigseries[i,:,NE:]),axis=1)
        lambda_reconstruct[i,2,j,:]=np.sum(np.squeeze(meigseries[i,:,:])*np.squeeze(neigseries[i,:,:]),axis=1)

In [964]:
xticks = np.linspace(0.0,1.0,3)
xlims = [-0.1,1.1]
yticklambda = np.linspace(-5,5,3)
ylimlambda = [-5.1,5.1]
ytickpc = np.linspace(0.5,1.0,2) # homogeneous
# ytickpc = np.linspace(0.70,1.0,2) # RII
ylimpc = [0.49,1.01] # homogeneous
# ylimpc = [0.69,1.01] # RII
fig,ax=plt.subplots(1,3,figsize=(12,3))
# FIGURE 1
for i in range(ngavg):
    for j in range(nrank):
        # RECONSTRUCT EIGENVALUES
        # m1
        ax[0].scatter(gaverageseries[i]*np.ones(nbatch),lambda_reconstruct[i,0,j,:],s=2,color='orange',alpha=0.5)
        ax[0].scatter(gaverageseries[i]*np.ones(nbatch),lambda_reconstruct[i,1,j,:],s=2,color='cyan',alpha=0.5)
        ax[0].scatter(gaverageseries[i]*np.ones(nbatch),lambda_reconstruct[i,2,j,:],s=2,color='gray',alpha=0.5)

        
        # OVERLAP
        # m1
        # ax[j][1].scatter(gaverageseries[i]*np.ones(nbatch),overlapeig[i,0,j,:],s=5,color='red',alpha=0.5)
        # ax[j][1].scatter(gaverageseries[i]*np.ones(nbatch),overlapeig[i,1,j,:],s=5,color='blue',alpha=0.5)
        ax[1].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(overlapeig_renorm[i,0,j,:]),s=2,color='orange',alpha=0.5)
        ax[1].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(overlapeig_renorm[i,1,j,:]),s=2,color='cyan',alpha=0.5)
        ax[1].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(overlapeig_renorm[i,2,j,:]),s=2,color='gray',alpha=0.5)


        # n1
        ax[2].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(overlapeig_renorm[i,0,j+nrank,:]),s=2,color='orange',alpha=0.5)
        ax[2].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(overlapeig_renorm[i,1,j+nrank,:]),s=2,color='cyan',alpha=0.5)
        ax[2].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(overlapeig_renorm[i,2,j+nrank,:]),s=2,color='gray',alpha=0.5)

for j in range(nrank):
    lambdaE = np.mean(np.squeeze(lambda_reconstruct[:,0,j,:]),axis=1)
    lambdaI = np.mean(np.squeeze(lambda_reconstruct[:,1,j,:]),axis=1)
    lambdaT = np.mean(np.squeeze(lambda_reconstruct[:,2,j,:]),axis=1)
    ax[0].plot(gaverageseries,lambdaE,color='red',linewidth=1.5,alpha=1.0,label='E population')
    ax[0].plot(gaverageseries,lambdaI,color='blue',linewidth=1.5,alpha=1.0,label='E population')
    ax[0].plot(gaverageseries,lambdaT,color='black',linewidth=1.5,alpha=1.0,label='E population')
    ax[0].set_xlim(xlims)
    ax[0].set_ylim(ylimlambda)
    ax[0].set_xticks(xticks)
    ax[0].set_yticks(yticklambda)

    mE = np.mean(np.squeeze(np.abs(overlapeig_renorm[:,0,j,:])),axis=1)
    mI = np.mean(np.squeeze(np.abs(overlapeig_renorm[:,1,j,:])),axis=1)
    mT = np.mean(np.squeeze(np.abs(overlapeig_renorm[:,2,j,:])),axis=1)
    ax[1].plot(gaverageseries,mE,color='red',linewidth=1.5,alpha=1.0,label='E population')
    ax[1].plot(gaverageseries,mI,color='blue',linewidth=1.5,alpha=1.0,label='I population')
    ax[1].plot(gaverageseries,mT,color='black',linewidth=1.5,alpha=1.0,label='Total')
    ax[1].set_xlim(xlims)
    ax[1].set_ylim(ylimpc)
    ax[1].set_xticks(xticks)
    ax[1].set_yticks(ytickpc)

    nE = np.mean(np.squeeze(np.abs(overlapeig_renorm[:,0,j+nrank,:])),axis=1)
    nI = np.mean(np.squeeze(np.abs(overlapeig_renorm[:,1,j+nrank,:])),axis=1)
    nT = np.mean(np.squeeze(np.abs(overlapeig_renorm[:,2,j+nrank,:])),axis=1)
    ax[2].plot(gaverageseries,nE,color='red',linewidth=1.5,alpha=1.0,label='E population')
    ax[2].plot(gaverageseries,nI,color='blue',linewidth=1.5,alpha=1.0,label='I population')
    ax[2].plot(gaverageseries,nT,color='black',linewidth=1.5,alpha=1.0,label='Total')
    ax[2].set_xlim(xlims)
    ax[2].set_ylim(ylimpc)
    ax[2].set_xticks(xticks)
    ax[2].set_yticks(ytickpc)
ax[2].legend()

In [5]:
# MEAN
xticks = np.linspace(0.0,1.0,3)
xlims = [-0.1,1.1]
ytickm = np.linspace(0.015,0.025,3)
ylimm = [0.015,0.025]
ytickn = np.linspace(0.00,0.20,3)
ylimn = [-0.01,0.20]
fig,ax=plt.subplots(2,3,figsize=(12,8))
# FIGURE same as overlap
idxax=0
for j in range(nrank):
    mmeanE = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:,0,j,:])),axis=1)
    mmeanI = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:,1,j,:])),axis=1)
    ax[0][idxax].plot(gaverageseries,mmeanE,color='red',linewidth=1.5,alpha=1.0,label='E population')
    ax[0][idxax].plot(gaverageseries,mmeanI,color='blue',linewidth=1.5,alpha=1.0,label='I population')
    ax[0][idxax].set_xlim(xlims)
    ax[0][idxax].set_ylim(ylimm)
    ax[0][idxax].set_xticks(xticks)
    ax[0][idxax].set_yticks(ytickm)

    nmeanE = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:,0,j+nrank,:])),axis=1)
    nmeanI = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:,1,j+nrank,:])),axis=1)
    ax[1][idxax].plot(gaverageseries,nmeanE,color='red',linewidth=1.5,alpha=1.0,label='E population')
    ax[1][idxax].plot(gaverageseries,nmeanI,color='blue',linewidth=1.5,alpha=1.0,label='I population')
    ax[1][idxax].set_xlim(xlims)
    ax[1][idxax].set_ylim(ylimn)
    ax[1][idxax].set_xticks(xticks)
    ax[1][idxax].set_yticks(ytickn)

# std
xticks = np.linspace(0.0,1.0,3)
xlims = [-0.1,1.1]
ytickm = np.linspace(0.00,0.02,3)
ylimm = [-0.001,0.02]
ytickn = np.linspace(0.00,0.20,3)
ylimn = [-0.01,0.20]
fig,ax=plt.subplots(2,3,figsize=(12,8))
# FIGURE same as overlap
idxax=2
for j in range(nrank):
    mstdE = np.mean(np.squeeze(np.abs(stdloadingeigclouds[:,0,j,:])),axis=1)
    mstdI = np.mean(np.squeeze(np.abs(stdloadingeigclouds[:,1,j,:])),axis=1)
    ax[0][idxax].plot(gaverageseries,mstdE,color='red',linewidth=1.5,alpha=1.0,label='E population')
    ax[0][idxax].plot(gaverageseries,mstdI,color='blue',linewidth=1.5,alpha=1.0,label='I population')
    ax[0][idxax].set_xlim(xlims)
    ax[0][idxax].set_ylim(ylimm)
    ax[0][idxax].set_xticks(xticks)
    ax[0][idxax].set_yticks(ytickm)

    nstdE = np.mean(np.squeeze(np.abs(stdloadingeigclouds[:,0,j+nrank,:])),axis=1)
    nstdI = np.mean(np.squeeze(np.abs(stdloadingeigclouds[:,1,j+nrank,:])),axis=1)
    ax[1][idxax].plot(gaverageseries,nstdE,color='red',linewidth=1.5,alpha=1.0,label='E population')
    ax[1][idxax].plot(gaverageseries,nstdI,color='blue',linewidth=1.5,alpha=1.0,label='I population')
    ax[1][idxax].set_xlim(xlims)
    ax[1][idxax].set_ylim(ylimn)
    ax[1][idxax].set_xticks(xticks)
    ax[1][idxax].set_yticks(ytickn)

In [6]:
ytickavg = np.linspace(0.020,0.032,3)
ylimavg = [0.0199,0.0321]
fig,ax=plt.subplots(1,2,figsize=(12,3))
# FIGURE same as overlap
for i in range(ngavg):
    for j in range(nrank):
        # mean m1
        ax[0].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,0,j,:])/np.sqrt(Norm2[i,0,j,:]),s=5,color='orange',alpha=0.25)
        ax[0].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,1,j,:])/np.sqrt(Norm2[i,1,j,:]),s=5,color='cyan',alpha=0.25)
        # mean n1
        ax[1].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,0,nrank+j,:])/np.sqrt(Norm2[i,0,j+nrank,:]),s=5,color='orange',alpha=0.25)
        ax[1].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,1,nrank+j,:])/np.sqrt(Norm2[i,1,j+nrank,:]),s=5,color='cyan',alpha=0.25)

for j in range(nrank):
    mmeanE = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:,0,j,:]))/np.squeeze(np.sqrt(Norm2[:,0,j,:])),axis=1)
    mmeanI = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:,1,j,:]))/np.squeeze(np.sqrt(Norm2[:,1,j,:])),axis=1)
    ax[0].plot(gaverageseries,mmeanE,color='red',linewidth=1.5,alpha=1.0,label='E population')
    ax[0].plot(gaverageseries,mmeanI,color='blue',linewidth=1.5,alpha=1.0,label='I population')
    ax[0].set_xlim(xlims)
    ax[0].set_ylim(ylimavg)
    ax[0].set_xticks(xticks)
    ax[0].set_yticks(ytickavg)

    nmeanE = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:,0,j+nrank,:]))/np.squeeze(np.sqrt(Norm2[:,0,j+nrank,:])),axis=1)
    nmeanI = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:,1,j+nrank,:]))/np.squeeze(np.sqrt(Norm2[:,1,j+nrank,:])),axis=1)
    ax[1].plot(gaverageseries,nmeanE,color='red',linewidth=1.5,alpha=1.0,label='E population')
    ax[1].plot(gaverageseries,nmeanI,color='blue',linewidth=1.5,alpha=1.0,label='I population')
    ax[1].set_xlim(xlims)
    ax[1].set_ylim(ylimavg)
    ax[1].set_xticks(xticks)
    ax[1].set_yticks(ytickavg)
ax[1].legend()
fig,ax=plt.subplots(1,2,figsize=(12,3))
# FIGURE absolute value
for i in range(ngavg):
    for j in range(nrank):
        # mean m1
        ax[0].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,0,j,:]),s=5,color='red',alpha=0.5)
        ax[0].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,1,j,:]),s=5,color='blue',alpha=0.5)
        # mean n1
        ax[1].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,0,nrank+j,:]),s=5,color='red',alpha=0.5)
        ax[1].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,1,nrank+j,:]),s=5,color='blue',alpha=0.5)

In [8]:
# statistics
mAm2,nAm2=np.zeros(2),np.zeros(2)
for i in range(nrank):
    mAm2[:]=meigvecAm[1::NE,i]
    nAm2[:]=neigvecAm[1::NE,i]
print('rank-1',mAm2,nAm2)
meanloadingeig_trialsavg=np.zeros((ngavg,2,2*nrank))
# m1--
meanloadingeig_trialsavg[:,:,0] = np.mean(np.abs(np.squeeze(meanloadingeigclouds[:,:,0,:])),axis=2)#np.squeeze(meanloadingeigclouds[:,:,:,12])#
meanloadingeig_trialsavg[:,:,1] = np.mean(np.abs(np.squeeze(meanloadingeigclouds[:,:,1,:])),axis=2)
meanloadingeig_trialsavg[:,1,1]*=-1
print('mean:',np.shape(meanloadingeig_trialsavg))
Oeigseries=np.zeros((ngavg,2,nrank*2))
for i in range(ngavg):
    m_mean = np.squeeze(meanloadingeig_trialsavg[i,:,:nrank])
    n_mean = np.squeeze(meanloadingeig_trialsavg[i,:,nrank:])
    print('num',m_mean,n_mean)
    Oeigm,Oeign = m_mean[:]/mAm2[:],n_mean[:]/nAm2[:]
    Oeigseries[i,:,:nrank],Oeigseries[i,:,nrank:]=np.reshape(Oeigm.copy(),(2,nrank)),np.reshape(Oeign.copy(),(2,nrank))

# # three copy
# idx = 18
# grand = gaverageseries[idx]*np.ones(4)*np.sqrt(2.0/3.0)
# grand[0]=0
# xee,xei,xie,xii=grand[0],grand[1],grand[2],grand[3]
# gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
# eigvgm,eigvecgm=la.eig(gmat)
# # print(eigvgm)
# print(Oeigseries[idx,:,:nrank])
# print(Oeigseries[idx,:,nrank:])
# attt=(1-gaverageseries[idx]**2/eigvAm[:nrank]**2)#eigvgm**2/eigvAm[:nrank]**2)
# print(np.sqrt(attt))
fig,ax=plt.subplots(figsize=(5,5))
# FIGURE absolute value
ax.plot(gaverageseries,Oeigseries[:,0,0],color='red',linewidth=1.5,label=r'$\alpha_{m}^E$')
ax.plot(gaverageseries,Oeigseries[:,1,0],color='blue',linewidth=1.5,label=r'$\alpha_{m}^I$')
ax.plot(gaverageseries,Oeigseries[:,0,1],color='orange',linewidth=1.5,label=r'$\alpha_{n}^E$')
ax.plot(gaverageseries,Oeigseries[:,1,1],color='cyan',linewidth=1.5,label=r'$\alpha_{n}^I$')
ax.plot(gaverageseries,np.sqrt(1-gaverageseries**2/eigvAm[0]**2),color='black',linestyle='--',linewidth=1.5,label=r'$\alpha_m^{theo}$')
ax.plot(gaverageseries,1/np.sqrt(1-gaverageseries**2/eigvAm[0]**2),color='gray',linestyle='--',linewidth=1.5,label=r'$\alpha_n^{theo}$')
yalims=[0.89,1.11]
yaticks=np.linspace(0.9,1.1,3)
# ax.set_xlim(xlims)
# ax.set_xticks(xticks)
# ax.set_ylim(yalims)
# ax.set_yticks(yaticks)
# ax.legend()


rank-1 [0.02236068 0.02236068] [ 0.04472136 -0.15652476]
mean: (20, 2, 2)
num [0.02235903 0.02236022] [ 0.04472014 -0.15651718]
num [0.02235812 0.02235487] [ 0.04473336 -0.15654439]
num [0.02235169 0.02235056] [ 0.04473878 -0.15658316]
num [0.02234406 0.02234335] [ 0.04482076 -0.15663134]
num [0.02232828 0.02234011] [ 0.04479579 -0.15669568]
num [0.0223228  0.02232151] [ 0.04481626 -0.15678454]
num [0.02231595 0.02230202] [ 0.04478607 -0.15687706]
num [0.02230133 0.02228485] [ 0.04481012 -0.15698728]
num [0.02228842 0.02226135] [ 0.04486376 -0.15703347]
num [0.02224691 0.02226509] [ 0.04507566 -0.15723594]
num [0.02222314 0.02223946] [ 0.04494079 -0.15730409]
num [0.02222595 0.02219162] [ 0.0449819  -0.15756183]
num [0.02216873 0.02219467] [ 0.04501731 -0.15775777]
num [0.02214881 0.02216232] [ 0.04510592 -0.15787418]
num [0.0221215  0.02212697] [ 0.04510514 -0.15806673]
num [0.02211774 0.02205755] [ 0.04524784 -0.1583446 ]
num [0.02205397 0.02205814] [ 0.04518472 -0.15850164]
num [0.0

# RANK-2

In [921]:

''' RANK-2 '''
''' heterogeneous case with single zero g_{EE} '''
# strfile = '2021-01-20-10_38_16_heterogeneousgIIR2_cloudsdata'
# strfile = '2021-01-20-11_12_48_homogeneousR2_cloudsdata'
# strfile ='2021-01-20-11_41_35_heterogeneousNotgEER2_cloudsdata'
strfile='2021-01-23-16_18_40_heterogeneousNotgEER2_cloudsdata'
# strfile='2021-01-23-17_02_52_homogeneousR2_cloudsdata'
# strfile = '2021-01-24-20_42_28_heterogeneousrIIR2_cloudsdata'
data = np.load(strfile+'.npz')#'2021-01-19-12_02_53_heterogeneousgII_cloudsdata.npz')
meigseries,neigseries=data['meigseries'],data['neigseries']
gaverageseries=data['gaverageseries']
# # loading calculated mean and std (2*2) for rank1, 0,0 m,E 0,1 m,I
# meanloadingseries,stdloadingseries=data['meanloadingseries'],data['stdloadingseries']
# ((niter,ntrials,NE*2))
ngavg,nbatch,_,_=np.shape(meigseries)
nrank=2
Jparameters=data['Jparameters']
NE = int(Jparameters[4])

P = np.zeros((2,2))
P = np.array([[Jparameters[0],-Jparameters[1]],[Jparameters[2],-Jparameters[3]]])
Am=np.zeros((NE*2,NE*2))
Am[:NE,:NE],Am[:NE,NE:]=P[0,0]/NE,P[0,1]/NE
Am[NE:,:NE],Am[NE:,NE:]=P[1,0]/NE,P[1,1]/NE
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
print('\bar{m}&\bar{n}')
print(meigvecAm[1::NE,0],meigvecAm[1::NE,1])
print(neigvecAm[1::NE,0],neigvecAm[1::NE,1])
print('eigenvalues:',eigvAm[:nrank])
print(np.sum(neigvecAm[:,0]*meigvecAm[:,0]))
print(np.sum(neigvecAm[:,1]*meigvecAm[:,1]))
Jrecon=np.reshape(meigvecAm[:,0],(NE*2,1))@np.reshape(neigvecAm[:,0],(1,NE*2))+np.reshape(meigvecAm[:,1],(NE*2,1))@np.reshape(neigvecAm[:,1],(1,NE*2))
print(Jrecon[1::NE,1::NE])
print(Am[1::NE,1::NE])

ar{m}ar{n}
[-0.01661519+0.j -0.02690605+0.j] [0.02650294+0.j 0.01725092+0.j]
[-0.19787336+0.j  0.30399677-0.j] [-0.08254572+0.j  0.05097416-0.j]
eigenvalues: [-4.89164729+0.j -1.30835271+0.j]
(-4.8916472867170295+0j)
(-1.3083527132830093+0j)
[[ 0.0011+0.j -0.0037+0.j]
 [ 0.0039+0.j -0.0073+0.j]]
[[ 0.0011 -0.0037]
 [ 0.0039 -0.0073]]


## [Fig.] choose the data for m and n, delete data that are far away from the standard range

In [811]:
meigvecloading,neigvecloading=data['meigseries'],data['neigseries']

idxgavg,idxtrial=18,7#10
# for igavg in range(ngavg):
#     # rank1
#     irank=0
#     meanmE = np.mean(np.squeeze(meigvecloading[igavg,idxtrial,:NE,irank]))#,axis=1)#np.zeros((ngavg,2,nrank*2,nbatch)),
#     # print(np.shape(meanmE))
#     idxmE =np.where(meanmE>0)
#     meigvecloading[igavg,idxmE,:NE]*=-1
#     meanmI = np.mean(np.squeeze(meigvecloading[igavg,idxtrial,NE:,irank]))#,axis=1)
#     idxmI =np.where(meanmI>0)
#     meigvecloading[igavg,idxmI,NE:]*=-1

#     meannE = np.mean(np.squeeze(neigvecloading[igavg,idxtrial,:NE,irank]))#,axis=1)#np.zeros((ngavg,2,nrank*2,nbatch)),
#     idxnE =np.where(meannE>0)
#     neigvecloading[igavg,idxnE,:NE]*=-1
#     meannI = np.mean(np.squeeze(neigvecloading[igavg,idxtrial,NE:,irank]))#,axis=1)
#     idxnI =np.where(meannI<0)
#     neigvecloading[igavg,idxnI,NE:]*=-1
#     # rank2
#     irank+=1
#     meanmE = np.mean(np.squeeze(meigvecloading[igavg,idxtrial,:NE,irank]))#,axis=1)#np.zeros((ngavg,2,nrank*2,nbatch)),
#     # print(np.shape(meanmE))
#     idxmE =np.where(meanmE<0)
#     meigvecloading[igavg,idxmE,:NE]*=-1
#     meanmI = np.mean(np.squeeze(meigvecloading[igavg,idxtrial,NE:,irank]))#,axis=1)
#     idxmI =np.where(meanmI<0)
#     meigvecloading[igavg,idxmI,NE:]*=-1

#     meannE = np.mean(np.squeeze(neigvecloading[igavg,idxtrial,:NE,irank]))#,axis=1)#np.zeros((ngavg,2,nrank*2,nbatch)),
#     idxnE =np.where(meannE>0)
#     neigvecloading[igavg,idxnE,:NE]*=-1
#     meannI = np.mean(np.squeeze(neigvecloading[igavg,idxtrial,NE:,irank]))#,axis=1)
#     idxnI =np.where(meannI<0)
#     neigvecloading[igavg,idxnI,NE:]*=-1
## puring
meanmE,meanmI,meannE,meannI=np.zeros((ngavg,nrank)),np.zeros((ngavg,nrank)),np.zeros((ngavg,nrank)),np.zeros((ngavg,nrank))
nbatch=1
for igavg in range(ngavg):
    for irank in range(nrank):
        # puring E
        mEmat = np.squeeze(meigvecloading[igavg,idxtrial,:NE,irank])
        meanmEtotal,stdmEtotal = np.mean(np.mean(mEmat)),np.std(np.reshape(mEmat,(NE*nbatch)))
        varmE = mEmat - meanmEtotal
        idxwhere = np.where(np.abs(varmE)>3.0*stdmEtotal)
        mEmat[idxwhere]=meanmEtotal
        meigvecloading[igavg,idxtrial,:NE,irank]=mEmat
        meanmE[igavg,irank]=np.mean(np.mean(mEmat))

        # puring I
        mImat = np.squeeze(meigvecloading[igavg,idxtrial,NE:,irank])
        meanmItotal,stdmItotal = np.mean(np.mean(mImat)),np.std(np.reshape(mImat,(NE*nbatch)))
        varmI = mImat - meanmItotal
        idxwhere = np.where(np.abs(varmI)>3.0*stdmItotal)
        mImat[idxwhere]=meanmItotal
        meigvecloading[igavg,idxtrial,NE:,irank]=mImat
        meanmI[igavg,irank]=np.mean(np.mean(mImat))
        
        # puring E
        nEmat = np.squeeze(neigvecloading[igavg,idxtrial,:NE,irank])
        meannEtotal,stdnEtotal = np.mean(np.mean(nEmat)),np.std(np.reshape(nEmat,(NE*nbatch)))
        varnE = nEmat - meannEtotal
        idxwhere = np.where(np.abs(varnE)>3.0*stdnEtotal)
        nEmat[idxwhere]=meannEtotal
        neigvecloading[igavg,idxtrial,:NE,irank]=nEmat
        meannE[igavg,irank]=np.mean(np.mean(nEmat))

        # puring I
        nImat = np.squeeze(neigvecloading[igavg,idxtrial,NE:,irank])
        meannItotal,stdnItotal = np.mean(np.mean(nImat)),np.std(np.reshape(nImat,(NE*nbatch)))
        varnI = nImat - meannItotal
        idxwhere = np.where(np.abs(varnI)>3.0*stdnItotal)
        nImat[idxwhere]=meannItotal
        neigvecloading[igavg,idxtrial,NE:,irank]=nImat
        meannI[igavg,irank]=np.mean(np.mean(nImat))

''' calculate directions '''
noiseE,noiseI = np.zeros((ngavg,NE*nbatch,2,4)),np.zeros((ngavg,NE*nbatch,2,4))
dirvecE,dirvecI=np.zeros((ngavg,2,2,4)),np.zeros((ngavg,2,2,4))
for i in range(ngavg):
    idtype=0
    # m1n1
    noiseE[i,:,0,idtype],noiseI[i,:,0,idtype],noiseE[i,:,1,idtype],noiseI[i,:,1,idtype]=np.squeeze(np.reshape(np.squeeze(meigvecloading[i,idxtrial,:NE,0]),(NE*nbatch,1)))-meanmE[i,0],np.squeeze(np.reshape(np.squeeze(meigvecloading[i,idxtrial,NE:,0]),(NE*nbatch,1)))-meanmI[i,0],np.squeeze(np.reshape(np.squeeze(neigvecloading[i,idxtrial,:NE,0]),(NE*nbatch,1)))-meannE[i,0],np.squeeze(np.reshape(np.squeeze(neigvecloading[i,idxtrial,NE:,0]),(NE*nbatch,1)))-meannI[i,0]
    covdirE,covdirI=np.squeeze(noiseE[i,:,:,idtype]).T@np.squeeze(noiseE[i,:,:,idtype]),np.squeeze(noiseI[i,:,:,idtype]).T@np.squeeze(noiseI[i,:,:,idtype])
    _,dirvecE[i,:,:,idtype]=la.eig(covdirE)
    _,dirvecI[i,:,:,idtype]=la.eig(covdirI)
    idtype+=1
    # m1n2
    noiseE[i,:,0,idtype],noiseI[i,:,0,idtype],noiseE[i,:,1,idtype],noiseI[i,:,1,idtype]=np.squeeze(np.reshape(np.squeeze(meigvecloading[i,idxtrial,:NE,0]),(NE*nbatch,1)))-meanmE[i,0],np.squeeze(np.reshape(np.squeeze(meigvecloading[i,idxtrial,NE:,0]),(NE*nbatch,1)))-meanmI[i,0],np.squeeze(np.reshape(np.squeeze(neigvecloading[i,idxtrial,:NE,1]),(NE*nbatch,1)))-meannE[i,1],np.squeeze(np.reshape(np.squeeze(neigvecloading[i,idxtrial,NE:,1]),(NE*nbatch,1)))-meannI[i,1]
    # cov
    covdirE,covdirI=np.squeeze(noiseE[i,:,:,idtype]).T@np.squeeze(noiseE[i,:,:,idtype]),np.squeeze(noiseI[i,:,:,idtype]).T@np.squeeze(noiseI[i,:,:,idtype])
    _,dirvecE[i,:,:,idtype]=la.eig(covdirE)
    _,dirvecI[i,:,:,idtype]=la.eig(covdirI)
    idtype+=1
    # m2n1
    noiseE[i,:,0,idtype],noiseI[i,:,0,idtype],noiseE[i,:,1,idtype],noiseI[i,:,1,idtype]=np.squeeze(np.reshape(np.squeeze(meigvecloading[i,idxtrial,:NE,1]),(NE*nbatch,1)))-meanmE[i,1],np.squeeze(np.reshape(np.squeeze(meigvecloading[i,idxtrial,NE:,1]),(NE*nbatch,1)))-meanmI[i,1],np.squeeze(np.reshape(np.squeeze(neigvecloading[i,idxtrial,:NE,0]),(NE*nbatch,1)))-meannE[i,0],np.squeeze(np.reshape(np.squeeze(neigvecloading[i,idxtrial,NE:,0]),(NE*nbatch,1)))-meannI[i,0]
    # cov
    covdirE,covdirI=np.squeeze(noiseE[i,:,:,idtype]).T@np.squeeze(noiseE[i,:,:,idtype]),np.squeeze(noiseI[i,:,:,idtype]).T@np.squeeze(noiseI[i,:,:,idtype])
    _,dirvecE[i,:,:,idtype]=la.eig(covdirE)
    _,dirvecI[i,:,:,idtype]=la.eig(covdirI)
    idtype+=1
    # m2n2
    noiseE[i,:,0,idtype],noiseI[i,:,0,idtype],noiseE[i,:,1,idtype],noiseI[i,:,1,idtype]=np.squeeze(np.reshape(np.squeeze(meigvecloading[i,idxtrial,:NE,1]),(NE*nbatch,1)))-meanmE[i,1],np.squeeze(np.reshape(np.squeeze(meigvecloading[i,idxtrial,NE:,1]),(NE*nbatch,1)))-meanmI[i,1],np.squeeze(np.reshape(np.squeeze(neigvecloading[i,idxtrial,:NE,1]),(NE*nbatch,1)))-meannE[i,1],np.squeeze(np.reshape(np.squeeze(neigvecloading[i,idxtrial,NE:,1]),(NE*nbatch,1)))-meannI[i,1]
    # cov
    covdirE,covdirI=np.squeeze(noiseE[i,:,:,idtype]).T@np.squeeze(noiseE[i,:,:,idtype]),np.squeeze(noiseI[i,:,:,idtype]).T@np.squeeze(noiseI[i,:,:,idtype])
    _,dirvecE[i,:,:,idtype]=la.eig(covdirE)
    _,dirvecI[i,:,:,idtype]=la.eig(covdirI)


xlims =[-0.51,0.51]
xticks = np.linspace(-0.5,0.5,3)
ylims =[-0.11,0.11]
yticks = np.linspace(-0.1,0.1,3)

# figure for eigenvectors
fig,ax=plt.subplots(2,2,figsize=(12,4))
#m1n1
jmm,inn=0,0
ax[jmm][inn].scatter(neigvecloading[idxgavg,idxtrial,NE:,inn],meigvecloading[idxgavg,idxtrial,NE:,jmm],s=5.0,c='blue',alpha=0.5)
ax[jmm][inn].scatter(neigvecloading[idxgavg,idxtrial,:NE,inn],meigvecloading[idxgavg,idxtrial,:NE,jmm],s=5.0,c='red',alpha=0.5)
ax[jmm][inn].set_xlim(xlims)
ax[jmm][inn].set_xticks(xticks)
ax[jmm][inn].set_ylim(ylims)
ax[jmm][inn].set_yticks(yticks)
ax[jmm][inn].grid()
ax[jmm][inn].plot([meannE[idxgavg,inn],meannE[idxgavg,inn]+dirvecE[idxgavg,0,0,0]],[meanmE[idxgavg,jmm],meanmE[idxgavg,jmm]+dirvecE[idxgavg,1,0,0]],color='r',linestyle='--',linewidth=1.5) # final type
ax[jmm][inn].plot([meannE[idxgavg,inn],meannE[idxgavg,inn]+dirvecE[idxgavg,0,1,0]],[meanmE[idxgavg,jmm],meanmE[idxgavg,jmm]+dirvecE[idxgavg,1,1,0]],color='r',linestyle='--',linewidth=1.5)
ax[jmm][inn].plot([meannI[idxgavg,inn],meannI[idxgavg,inn]+dirvecI[idxgavg,0,0,0]],[meanmI[idxgavg,jmm],meanmI[idxgavg,jmm]+dirvecI[idxgavg,1,0,0]],color='b',linestyle='--',linewidth=1.5) # final type
ax[jmm][inn].plot([meannI[idxgavg,inn],meannI[idxgavg,inn]+dirvecI[idxgavg,0,1,0]],[meanmI[idxgavg,jmm],meanmI[idxgavg,jmm]+dirvecI[idxgavg,1,1,0]],color='b',linestyle='--',linewidth=1.5)

#m1n2
jmm,inn,it=0,1,1
ax[jmm][inn].scatter(neigvecloading[idxgavg,idxtrial,NE:,inn],meigvecloading[idxgavg,idxtrial,NE:,jmm],s=5.0,c='blue',alpha=0.5)
ax[jmm][inn].scatter(neigvecloading[idxgavg,idxtrial,:NE,inn],meigvecloading[idxgavg,idxtrial,:NE,jmm],s=5.0,c='red',alpha=0.5)
ax[jmm][inn].set_xlim(xlims)
ax[jmm][inn].set_xticks(xticks)
ax[jmm][inn].set_ylim(ylims)
ax[jmm][inn].set_yticks(yticks)
ax[jmm][inn].grid()
ax[jmm][inn].plot([meannE[idxgavg,inn],meannE[idxgavg,inn]+dirvecE[idxgavg,0,0,it]],[meanmE[idxgavg,jmm],meanmE[idxgavg,jmm]+dirvecE[idxgavg,1,0,it]],color='r',linestyle='--',linewidth=1.5) # final type
ax[jmm][inn].plot([meannE[idxgavg,inn],meannE[idxgavg,inn]+dirvecE[idxgavg,0,1,it]],[meanmE[idxgavg,jmm],meanmE[idxgavg,jmm]+dirvecE[idxgavg,1,1,it]],color='r',linestyle='--',linewidth=1.5)
ax[jmm][inn].plot([meannI[idxgavg,inn],meannI[idxgavg,inn]+dirvecI[idxgavg,0,0,it]],[meanmI[idxgavg,jmm],meanmI[idxgavg,jmm]+dirvecI[idxgavg,1,0,it]],color='b',linestyle='--',linewidth=1.5) # final type
ax[jmm][inn].plot([meannI[idxgavg,inn],meannI[idxgavg,inn]+dirvecI[idxgavg,0,1,it]],[meanmI[idxgavg,jmm],meanmI[idxgavg,jmm]+dirvecI[idxgavg,1,1,it]],color='b',linestyle='--',linewidth=1.5)

#m2n1
jmm,inn,it=1,0,2
ax[jmm][inn].scatter(neigvecloading[idxgavg,idxtrial,NE:,inn],meigvecloading[idxgavg,idxtrial,NE:,jmm],s=5.0,c='blue',alpha=0.5)
ax[jmm][inn].scatter(neigvecloading[idxgavg,idxtrial,:NE,inn],meigvecloading[idxgavg,idxtrial,:NE,jmm],s=5.0,c='red',alpha=0.5)
ax[jmm][inn].set_xlim(xlims)
ax[jmm][inn].set_xticks(xticks)
ax[jmm][inn].set_ylim(ylims)
ax[jmm][inn].set_yticks(yticks)
ax[jmm][inn].grid()
ax[jmm][inn].plot([meannE[idxgavg,inn],meannE[idxgavg,inn]+dirvecE[idxgavg,0,0,it]],[meanmE[idxgavg,jmm],meanmE[idxgavg,jmm]+dirvecE[idxgavg,1,0,it]],color='r',linestyle='--',linewidth=1.5) # final type
ax[jmm][inn].plot([meannE[idxgavg,inn],meannE[idxgavg,inn]+dirvecE[idxgavg,0,1,it]],[meanmE[idxgavg,jmm],meanmE[idxgavg,jmm]+dirvecE[idxgavg,1,1,it]],color='r',linestyle='--',linewidth=1.5)
ax[jmm][inn].plot([meannI[idxgavg,inn],meannI[idxgavg,inn]+dirvecI[idxgavg,0,0,it]],[meanmI[idxgavg,jmm],meanmI[idxgavg,jmm]+dirvecI[idxgavg,1,0,it]],color='b',linestyle='--',linewidth=1.5) # final type
ax[jmm][inn].plot([meannI[idxgavg,inn],meannI[idxgavg,inn]+dirvecI[idxgavg,0,1,it]],[meanmI[idxgavg,jmm],meanmI[idxgavg,jmm]+dirvecI[idxgavg,1,1,it]],color='b',linestyle='--',linewidth=1.5)

#m2n2
jmm,inn,it=1,1,3
ax[jmm][inn].scatter(neigvecloading[idxgavg,idxtrial,NE:,inn],meigvecloading[idxgavg,idxtrial,NE:,jmm],s=5.0,c='blue',alpha=0.5)
ax[jmm][inn].scatter(neigvecloading[idxgavg,idxtrial,:NE,inn],meigvecloading[idxgavg,idxtrial,:NE,jmm],s=5.0,c='red',alpha=0.5)
ax[jmm][inn].set_xlim(xlims)
ax[jmm][inn].set_xticks(xticks)
ax[jmm][inn].set_ylim(ylims)
ax[jmm][inn].set_yticks(yticks)
ax[jmm][inn].grid()
ax[jmm][inn].plot([meannE[idxgavg,inn],meannE[idxgavg,inn]+dirvecE[idxgavg,0,0,it]],[meanmE[idxgavg,jmm],meanmE[idxgavg,jmm]+dirvecE[idxgavg,1,0,it]],color='r',linestyle='--',linewidth=1.5) # final type
ax[jmm][inn].plot([meannE[idxgavg,inn],meannE[idxgavg,inn]+dirvecE[idxgavg,0,1,it]],[meanmE[idxgavg,jmm],meanmE[idxgavg,jmm]+dirvecE[idxgavg,1,1,it]],color='r',linestyle='--',linewidth=1.5)
ax[jmm][inn].plot([meannI[idxgavg,inn],meannI[idxgavg,inn]+dirvecI[idxgavg,0,0,it]],[meanmI[idxgavg,jmm],meanmI[idxgavg,jmm]+dirvecI[idxgavg,1,0,it]],color='b',linestyle='--',linewidth=1.5) # final type
ax[jmm][inn].plot([meannI[idxgavg,inn],meannI[idxgavg,inn]+dirvecI[idxgavg,0,1,it]],[meanmI[idxgavg,jmm],meanmI[idxgavg,jmm]+dirvecI[idxgavg,1,1,it]],color='b',linestyle='--',linewidth=1.5)


## OVERLAP

In [812]:
xlims =[-.1,1.1]
xticks = np.linspace(0,1.0,3)
ylims =[-10.1,5.1]
yticks = np.linspace(-10.,5.,3)
ylims1 =[-5.1,5.1]
yticks1 = np.linspace(-5.,5.,3)

overlaptt,overlapmean=np.zeros((ngavg,2,2,3)) ,np.zeros((ngavg,2,2,3)) # rank2,rank2,EITOTAL
# figure for eigenvectors
fig,ax=plt.subplots(4,1,figsize=(4,4))
#m1n1
for igavg in range(ngavg):
    for jmm in range(nrank):
        for inn in range(nrank):
            # total
            overlaptt[igavg,jmm,inn,0]=np.sum(np.squeeze(neigvecloading[idxgavg,idxtrial,:NE,inn])*np.squeeze(meigvecloading[idxgavg,idxtrial,:NE,jmm]))
            overlaptt[igavg,jmm,inn,1]=np.sum(np.squeeze(neigvecloading[idxgavg,idxtrial,NE:,inn])*np.squeeze(meigvecloading[idxgavg,idxtrial,NE:,jmm]))
            overlaptt[igavg,jmm,inn,2]=np.sum(np.squeeze(neigvecloading[idxgavg,idxtrial,:,inn])*np.squeeze(meigvecloading[idxgavg,idxtrial,:,jmm]))
            # mean
            overlapmean[igavg,jmm,inn,0]=np.squeeze(meannE[idxgavg,inn])*np.squeeze(meanmE[idxgavg,jmm])*NE
            overlapmean[igavg,jmm,inn,1]=np.squeeze(meannI[idxgavg,inn])*np.squeeze(meanmI[idxgavg,jmm])*NE
            overlapmean[igavg,jmm,inn,2]=np.squeeze(meannE[idxgavg,inn])*np.squeeze(meanmE[idxgavg,jmm])*NE+np.squeeze(meannI[idxgavg,inn])*np.squeeze(meanmI[idxgavg,jmm])*NE

for jmm in range(nrank):
    for inn in range(nrank):
        ijmn = jmm*(nrank)+inn
        ax[ijmn].plot(gaverageseries,overlaptt[:,jmm,inn,0],color='orange',linestyle='-',linewidth=2.5,alpha=0.5) # final type
        ax[ijmn].plot(gaverageseries,overlapmean[:,jmm,inn,0],color='red',linestyle='--',linewidth=1.5,alpha=1.0) # final type
        ax[ijmn].plot(gaverageseries,overlaptt[:,jmm,inn,1],color='cyan',linestyle='-',linewidth=2.5,alpha=0.5) # final type
        ax[ijmn].plot(gaverageseries,overlapmean[:,jmm,inn,1],color='blue',linestyle='--',linewidth=1.5,alpha=1.0) # final type
        ax[ijmn].plot(gaverageseries,overlaptt[:,jmm,inn,2],color='gray',linestyle='-',linewidth=2.5,alpha=0.5) # final type
        ax[ijmn].plot(gaverageseries,overlapmean[:,jmm,inn,2],color='black',linestyle='--',linewidth=1.5,alpha=1.0) # final type
# ax[0].set_xlim(xlims)
# ax[0].set_xticks([])#xticks)
# ax[0].set_ylim(ylims)
# ax[0].set_yticks(yticks)
# for i in np.arange(1,4,1):
#     ax[i].set_xlim(xlims)
#     # ax[i].set_xticks(xticks)
#     ax[i].set_ylim(ylims1)
#     ax[i].set_yticks(yticks1)
ax[3].set_xticks(xticks)



In [567]:
idxgavg=10
# generate figure
xticks = np.linspace(-6.0,1.0,5)
xlims = [-4.1,1.1]
yticks = np.linspace(-0.8,0.8,3)
ylims = [-0.81,0.81]
theta = np.linspace(0, 2 * np.pi, 200)
# fig,ax=plt.subplots(1,3,figsize=(12,4))
# ''' PLOT a sample scatter plot A '''
# ## three cases
# case one homogeneous
# strfile='2021-01-23-16_18_40_heterogeneousNotgEER2_cloudsdata'
strfile='2021-01-23-17_02_52_homogeneousR2_cloudsdata'
# strfile = '2021-01-24-20_42_28_heterogeneousrIIR2_cloudsdata'
data = np.load(strfile+'.npz')
gaverageseries=data['gaverageseries']
nrank=2
Jparameters=data['Jparameters']
JEE,JIE,JEI,JII=Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]
Nparams=np.array([NE,NE])
# generate mean 
Am,Jsv,ua,svam,va=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# OVERALL
eigvalAm,_=la.eig(Am)
# generate X
# totally average
gaverage=gaverageseries[idxgavg]
grand = gaverage*np.ones(4)/np.sqrt(2.0)
print('pre avg:',gaverage)
## homogeneous X matrix
X,Gamp,eigvgmT,eigvecgm,gaverage=generate_localstatsmat_eig(Nparams,grand)#,ugm,svgm,vgm
print('post avg:',gaverage)
xr = gaverage*np.cos(theta)
yr = gaverage*np.sin(theta)
# mix together
Xtotal=X.copy()+Am.copy()
eigvxt,eigvecxt=la.eig(Xtotal)
# eigenvalues
eigvalseries=data['eigvalseries']

fig,ax=plt.subplots(3,1,figsize=(8,8))
real_eigsbulk,imag_eigsbulk=np.real(eigvxt[2:]),np.imag(eigvxt[2:])
ax[0].scatter(real_eigsbulk,imag_eigsbulk,marker='^',s=5,color='blue',alpha=0.25)
ax[0].scatter(np.real(eigvxt[:2]),np.imag(eigvxt[:2]),marker='^',s=5,color='blue',alpha=0.5)
for i in range(nrank):
    ax[0].scatter(np.real(eigvalseries[idxgavg,:,i]),np.imag(eigvalseries[idxgavg,:,i]),marker='^',s=5,color='blue',alpha=0.5)
xa = gaverage*np.cos(theta)
ya = gaverage*np.sin(theta)
ax[0].plot(xa, ya, color="darkred", linewidth=1.5,label=r'$r_g$')
ax[0].plot(xr, yr, color="grey", linewidth=1.5,linestyle='--',label=r'fixed $\bar{g}$')
ax[0].scatter(np.real(eigvalAm[:2]),np.imag(eigvalAm[:2]),marker='^',s=15,color='red',alpha=1.0)
ax[0].set_xlim(xlims)
ax[0].set_ylim(ylims)
ax[0].set_xticks(xticks)
ax[0].set_yticks(yticks)
ax[0].set_aspect('equal')
ax[0].legend()
ax[0].grid()

# case two heterogeneous RII
# strfile='2021-01-23-16_18_40_heterogeneousNotgEER2_cloudsdata'
# strfile='2021-01-23-17_02_52_homogeneousR2_cloudsdata'
strfile = '2021-01-24-20_42_28_heterogeneousrIIR2_cloudsdata'
data = np.load(strfile+'.npz')
gaverageseries=data['gaverageseries']
# generate X
# max r_g
grand = np.zeros(4)
grand[3]=gaverage*np.sqrt(2) # I dominant
print('pre avg:',gaverage)
## homogeneous X matrix
X,Gamp,eigvgmT,eigvecgm,gaverage=generate_localstatsmat_eig(Nparams,grand)#,ugm,svgm,vgm
print('post avg:',gaverage)

# mix together
Xtotal=X.copy()+Am.copy()
eigvxt,eigvecxt=la.eig(Xtotal)
# eigenvalues
eigvalseries=data['eigvalseries']

real_eigsbulk,imag_eigsbulk=np.real(eigvxt[2:]),np.imag(eigvxt[2:])
ax[1].scatter(real_eigsbulk,imag_eigsbulk,marker='^',s=5,color='blue',alpha=0.25)
ax[1].scatter(np.real(eigvxt[:2]),np.imag(eigvxt[:2]),marker='^',s=5,color='blue',alpha=0.5)
for i in range(nrank):
    ax[1].scatter(np.real(eigvalseries[idxgavg,:,i]),np.imag(eigvalseries[idxgavg,:,i]),marker='^',s=5,color='blue',alpha=0.5)
xa = np.sqrt(2)*gaverage*np.cos(theta)
ya = np.sqrt(2)*gaverage*np.sin(theta)
ax[1].plot(xa, ya, color="darkred", linewidth=1.5,label=r'$r_g$')
ax[1].plot(xr, yr, color="grey", linewidth=1.5,linestyle='--',label=r'fixed $\bar{g}$')
ax[1].scatter(np.real(eigvalAm[:2]),np.imag(eigvalAm[:2]),marker='^',s=15,color='red',alpha=1.0)
ax[1].set_xlim(xlims)
ax[1].set_ylim(ylims)
ax[1].set_xticks(xticks)
ax[1].set_yticks(yticks)
ax[1].set_aspect('equal')
ax[0].set_aspect('equal')
ax[1].legend()
ax[1].grid()

# case two heterogeneous not gEE
strfile='2021-01-23-16_18_40_heterogeneousNotgEER2_cloudsdata'
# strfile='2021-01-23-17_02_52_homogeneousR2_cloudsdata'
# strfile = '2021-01-24-20_42_28_heterogeneousrIIR2_cloudsdata'
data = np.load(strfile+'.npz')
gaverageseries=data['gaverageseries']
# generate X
# three copy
grand = gaverage*np.ones(4)*np.sqrt(2.0/3.0)
grand[0]=0
print('pre avg:',gaverage)
## homogeneous X matrix
X,Gamp,eigvgmT,eigvecgm,gaverage=generate_localstatsmat_eig(Nparams,grand)#,ugm,svgm,vgm
print('post avg:',gaverage)

# mix together
Xtotal=X.copy()+Am.copy()
eigvxt,eigvecxt=la.eig(Xtotal)
# eigenvalues
eigvalseries=data['eigvalseries']

real_eigsbulk,imag_eigsbulk=np.real(eigvxt[2:]),np.imag(eigvxt[2:])
ax[2].scatter(real_eigsbulk,imag_eigsbulk,marker='^',s=5,color='blue',alpha=0.25)
ax[2].scatter(np.real(eigvxt[:2]),np.imag(eigvxt[:2]),marker='^',s=5,color='blue',alpha=0.5)
for i in range(nrank):
    ax[2].scatter(np.real(eigvalseries[idxgavg,:,i]),np.imag(eigvalseries[idxgavg,:,i]),marker='^',s=5,color='blue',alpha=0.5)
arg = np.sqrt((1+np.sqrt(5))/3.0)
xa = arg*gaverage*np.cos(theta)
ya = arg*gaverage*np.sin(theta)
ax[2].plot(xa, ya, color="darkred", linewidth=1.5,label=r'$r_g$')
ax[2].plot(xr, yr, color="grey", linewidth=1.5,linestyle='--',label=r'fixed $\bar{g}$')
ax[2].scatter(np.real(eigvalAm[:2]),np.imag(eigvalAm[:2]),marker='^',s=15,color='red',alpha=1.0)
ax[2].set_xlim(xlims)
ax[2].set_ylim(ylims)
ax[2].set_xticks(xticks)
ax[2].set_yticks(yticks)
ax[2].set_aspect('equal')
ax[0].set_aspect('equal')
ax[2].legend()
ax[2].grid()




pre avg: 0.5499999999999999
post avg: 0.5499999999999998
pre avg: 0.5499999999999998
post avg: 0.5499999999999998
pre avg: 0.5499999999999998
post avg: 0.5499999999999998


## eigenvalues of RANK-2

In [675]:
# generate figure
xticks = np.linspace(0.0,1.0,2)
xlims = [-0.1,1.1]
ytick2 = np.linspace(-1.50,-1.20,2)
ylim2 = [-1.51,-1.19]
ytick1 = np.linspace(-5.0,-4.7,2)
ylim1 = [-5.01,-4.69]
theta = np.linspace(0, 2 * np.pi, 200)
# fig,ax=plt.subplots(1,3,figsize=(12,4))
# ''' PLOT a sample scatter plot A '''
# ## three cases
# case one homogeneous
# strfile='2021-01-23-16_18_40_heterogeneousNotgEER2_cloudsdata'
strfile='2021-01-23-17_02_52_homogeneousR2_cloudsdata'
# strfile = '2021-01-24-20_42_28_heterogeneousrIIR2_cloudsdata'
data = np.load(strfile+'.npz')
gaverageseries=data['gaverageseries']
'''
nrank=2
Jparameters=data['Jparameters']
JEE,JIE,JEI,JII=Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]
Nparams=np.array([NE,NE])
# generate mean 
Am,Jsv,ua,svam,va=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
'''
# OVERALL
eigvalAm,_=la.eig(Am)
# eigenvalues
eigvalseries=data['eigvalseries']
# plot the single outliers   
fig,ax=plt.subplots(2,3,figsize=(9,3))
idxxx=0
for i in range(20):
    ax[0][idxxx].scatter(gaverageseries[i]*np.ones(20),eigvalseries[i,:,0],s=5.0, c='red', alpha=0.5)
    ax[1][idxxx].scatter(gaverageseries[i]*np.ones(20),eigvalseries[i,:,1],s=5.0, c='blue', alpha=0.5)
# numerical mean and std
meanlambda,stdlambda=np.zeros((ngavg,nrank)),np.zeros((ngavg,nrank))
for irank in range(nrank):
    meanlambda[:,irank]=np.mean(eigvalseries[:,:,irank],axis=1)
    stdlambda[:,irank]=np.std(eigvalseries[:,:,irank],axis=1)
ax[0][idxxx].plot(gaverageseries,meanlambda[:,0], lw=2, label=r'$\lambda_1$', color='red')
ax[0][idxxx].fill_between(gaverageseries,meanlambda[:,0]+stdlambda[:,0], meanlambda[:,0]-stdlambda[:,0], facecolor='orange', alpha=0.5)
ax[1][idxxx].plot(gaverageseries,meanlambda[:,1], lw=2, label=r'$\lambda_2$', color='blue')
ax[1][idxxx].fill_between(gaverageseries,meanlambda[:,1]+stdlambda[:,1], meanlambda[:,1]-stdlambda[:,1], facecolor='cyan', alpha=0.5)
ax[0][idxxx].plot(gaverageseries,eigvalAm[0]*np.ones(ngavg),'k--',linewidth=1.5,label=r'$\lambda_1^O$')
ax[1][idxxx].plot(gaverageseries,eigvalAm[1]*np.ones(ngavg),linestyle='--',color='gray',linewidth=1.5,label=r'$\lambda_2^O$')
for irank in range(nrank):
    ax[irank][idxxx].set_xlim(xlims)
    ax[irank][idxxx].set_xticks([])#xticks)#,fontsize=12)
    # ax[irank][idxxx].set_aspect('equal')
ax[irank][idxxx].set_xticks(xticks)
ax[0][idxxx].set_ylim(ylim1)
ax[0][idxxx].set_yticks(ytick1)#,fontsize=12)
ax[1][idxxx].set_ylim(ylim2)
ax[1][idxxx].set_yticks(ytick2)#,fontsize=12)
plt.rc('xtick',labelsize=9)
plt.rc('ytick',labelsize=9)
# ax.legend()#["mean outlier","original outlier"],fontsize=14)
    
''' case one hETEROGENEOUS RII '''
# strfile='2021-01-23-16_18_40_heterogeneousNotgEER2_cloudsdata'
# strfile='2021-01-23-17_02_52_homogeneousR2_cloudsdata'
strfile = '2021-01-24-20_42_28_heterogeneousrIIR2_cloudsdata'
data = np.load(strfile+'.npz')
# eigenvalues
eigvalseries=data['eigvalseries']
# plot the single outliers   
# fig,ax=plt.subplots(2,3,figsize=(9,3))
idxxx+=1
for i in range(15):
    ax[0][idxxx].scatter(gaverageseries[i]*np.ones(20),eigvalseries[i,:,0],s=5.0, c='red', alpha=0.5)
    ax[1][idxxx].scatter(gaverageseries[i]*np.ones(20),eigvalseries[i,:,1],s=5.0, c='blue', alpha=0.5)
# numerical mean and std
meanlambda,stdlambda=np.zeros((ngavg,nrank)),np.zeros((ngavg,nrank))
for irank in range(nrank):
    meanlambda[:,irank]=np.mean(eigvalseries[:,:,irank],axis=1)
    stdlambda[:,irank]=np.std(eigvalseries[:,:,irank],axis=1)
ax[0][idxxx].plot(gaverageseries[:15],meanlambda[:15,0], lw=2, label=r'$\lambda_1$', color='red')
ax[0][idxxx].fill_between(gaverageseries[:15],meanlambda[:15,0]+stdlambda[:15,0], meanlambda[:15,0]-stdlambda[:15,0], facecolor='orange', alpha=0.5)
ax[1][idxxx].plot(gaverageseries[:15],meanlambda[:15,1], lw=2, label=r'$\lambda_2$', color='blue')
ax[1][idxxx].fill_between(gaverageseries[:15],meanlambda[:15,1]+stdlambda[:15,1], meanlambda[:15,1]-stdlambda[:15,1], facecolor='cyan', alpha=0.5)
ax[0][idxxx].plot(gaverageseries[:],eigvalAm[0]*np.ones(ngavg),'k--',linewidth=1.5,label=r'$\lambda_1^O$')
ax[1][idxxx].plot(gaverageseries[:],eigvalAm[1]*np.ones(ngavg),linestyle='--',color='gray',linewidth=1.5,label=r'$\lambda_2^O$')
for irank in range(nrank):
    ax[irank][idxxx].set_xlim(xlims)
    ax[irank][idxxx].set_xticks([])#xticks)#,fontsize=12)
    # ax[irank][idxxx].set_aspect('equal')
ax[irank][idxxx].set_xticks(xticks)
ax[0][idxxx].set_ylim(ylim1)
ax[0][idxxx].set_yticks([])#ytick1)#,fontsize=12)
ax[1][idxxx].set_ylim(ylim2)
ax[1][idxxx].set_yticks([])#ytick2)#,fontsize=12)
plt.rc('xtick',labelsize=9)
plt.rc('ytick',labelsize=9)

''' case two heterogeneous NotrEE '''
strfile='2021-01-23-16_18_40_heterogeneousNotgEER2_cloudsdata'
# strfile='2021-01-23-17_02_52_homogeneousR2_cloudsdata'
# strfile = '2021-01-24-20_42_28_heterogeneousrIIR2_cloudsdata'
data = np.load(strfile+'.npz')
# eigenvalues
eigvalseries=data['eigvalseries']
# plot the single outliers   
# fig,ax=plt.subplots(2,3,figsize=(9,3))
idxxx+=1
for i in range(20):
    ax[0][idxxx].scatter(gaverageseries[i]*np.ones(20),eigvalseries[i,:,0],s=5.0, c='red', alpha=0.5)
    ax[1][idxxx].scatter(gaverageseries[i]*np.ones(20),eigvalseries[i,:,1],s=5.0, c='blue', alpha=0.5)
# numerical mean and std
meanlambda,stdlambda=np.zeros((ngavg,nrank)),np.zeros((ngavg,nrank))
for irank in range(nrank):
    meanlambda[:,irank]=np.mean(eigvalseries[:,:,irank],axis=1)
    stdlambda[:,irank]=np.std(eigvalseries[:,:,irank],axis=1)
ax[0][idxxx].plot(gaverageseries,meanlambda[:,0], lw=2, label=r'$\lambda_1$', color='red')
ax[0][idxxx].fill_between(gaverageseries,meanlambda[:,0]+stdlambda[:,0], meanlambda[:,0]-stdlambda[:,0], facecolor='orange', alpha=0.5)
ax[1][idxxx].plot(gaverageseries,meanlambda[:,1], lw=2, label=r'$\lambda_2$', color='blue')
ax[1][idxxx].fill_between(gaverageseries,meanlambda[:,1]+stdlambda[:,1], meanlambda[:,1]-stdlambda[:,1], facecolor='cyan', alpha=0.5)
ax[0][idxxx].plot(gaverageseries,eigvalAm[0]*np.ones(ngavg),'k--',linewidth=1.5,label=r'$\lambda_1^O$')
ax[1][idxxx].plot(gaverageseries,eigvalAm[1]*np.ones(ngavg),linestyle='--',color='gray',linewidth=1.5,label=r'$\lambda_2^O$')
for irank in range(nrank):
    ax[irank][idxxx].set_xlim(xlims)
    ax[irank][idxxx].set_xticks([])#xticks)#,fontsize=12)
    # ax[irank][idxxx].set_aspect('equal')
ax[irank][idxxx].set_xticks(xticks)
ax[0][idxxx].set_ylim(ylim1)
ax[0][idxxx].set_yticks([])#ytick1)#,fontsize=12)
ax[1][idxxx].set_ylim(ylim2)
ax[1][idxxx].set_yticks([])#ytick2)#,fontsize=12)
plt.rc('xtick',labelsize=9)
plt.rc('ytick',labelsize=9)

# ax[2].grid()




In [922]:
''' For rank 2 '''
# calculate \theta overlap
loadingeigvec=np.zeros((ngavg,2,nrank*2,NE,nbatch)) # ng,EI,nrank*2 mn,Npopulation,ntrials
loadingeigvecAm=np.zeros((2,nrank*2,NE))
meanloadingeigclouds,stdloadingeigclouds = np.zeros((ngavg,2,nrank*2,nbatch)),np.zeros((ngavg,2,nrank*2,nbatch))
Norm2=np.zeros((ngavg,3,nrank*2,nbatch))
Norm2Am=np.zeros((3,nrank*2,nbatch))
overlapeig,overlapeig_renorm=np.zeros((ngavg,3,nrank*2,nbatch)),np.zeros((ngavg,3,nrank*2,nbatch))
# lambda reconstruction
lambda_reconstruct=np.zeros((ngavg,3,nrank,nbatch)) # E pop, I pop, overall
for j in range(nrank):
    # eig
    loadingeigvecAm[0,j,:]=np.squeeze(meigvecAm[:NE,j]).T # m_j^E
    loadingeigvecAm[0,j+nrank,:]=np.squeeze(neigvecAm[:NE,j]).T # n_j^E
    loadingeigvecAm[1,j,:]=np.squeeze(meigvecAm[NE:,j]).T # m_j^I
    loadingeigvecAm[1,j+nrank,:]=np.squeeze(neigvecAm[NE:,j]).T # n_j^I
    Norm2Am[0,j]=np.sum(np.squeeze(loadingeigvecAm[0,j,:])**2,axis=0) # excitatory
    Norm2Am[1,j]=np.sum(np.squeeze(loadingeigvecAm[1,j,:])**2,axis=0) # inhibitory
    Norm2Am[2,j]=Norm2Am[1,j]+Norm2Am[0,j] # overall
    Norm2Am[0,j+nrank]=np.sum(np.squeeze(loadingeigvecAm[0,j+nrank,:])**2,axis=0) # excitatory
    Norm2Am[1,j+nrank]=np.sum(np.squeeze(loadingeigvecAm[1,j+nrank,:])**2,axis=0) # inhibitory
    Norm2Am[2,j+nrank]=Norm2Am[1,j+nrank]+Norm2Am[0,j+nrank] # overall
for j in range(nrank):
    for i in range(ngavg):
        # eig
        loadingeigvec[i,0,j,:,:]=np.squeeze(meigseries[i,:,:NE,j]).T # m_j^E
        loadingeigvec[i,0,j+nrank,:,:]=np.squeeze(neigseries[i,:,:NE,j]).T # n_j^E
        loadingeigvec[i,1,j,:,:]=np.squeeze(meigseries[i,:,NE:,j]).T # m_j^I
        loadingeigvec[i,1,j+nrank,:,:]=np.squeeze(neigseries[i,:,NE:,j]).T # n_j^I

        # renormalization
        # m, right eigenvectors
        Norm2[i,0,j,:]=np.sum(np.squeeze(loadingeigvec[i,0,j,:,:])**2,axis=0) # excitatory
        Norm2[i,1,j,:]=np.sum(np.squeeze(loadingeigvec[i,1,j,:,:])**2,axis=0) # inhibitory
        Norm2[i,2,j,:]=Norm2[i,1,j,:]+Norm2[i,0,j,:] # overall
        # n, left eigenvectors
        Norm2[i,0,j+nrank,:]=np.sum(np.squeeze(loadingeigvec[i,0,j+nrank,:,:])**2,axis=0) # excitatory n
        Norm2[i,1,j+nrank,:]=np.sum(np.squeeze(loadingeigvec[i,1,j+nrank,:,:])**2,axis=0) # inhibitory n
        Norm2[i,2,j+nrank,:]=Norm2[i,1,j+nrank,:]+Norm2[i,0,j+nrank,:] # overall normalization
        # overlap
        # m
        overlapeig[i,0,j,:]=np.squeeze(np.reshape(meigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j,:,:])) # excitatory overlap -- renormalized
        overlapeig[i,1,j,:]=np.squeeze(np.reshape(meigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j,:,:])) # inhibitory overlap -- renromalized
        overlapeig[i,2,j,:]=np.squeeze(np.reshape(meigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j,:,:]))+np.squeeze(np.reshape(meigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j,:,:])) # inhibitory overlap -- renormalized
        # n
        overlapeig[i,0,j+nrank,:]=np.squeeze(np.reshape(neigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j+nrank,:,:]))/(eigvAm[j])**2 # excitatory overlap -- renormalized
        overlapeig[i,1,j+nrank,:]=np.squeeze(np.reshape(neigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j+nrank,:,:]))/(eigvAm[j])**2# inhibitory overlap -- renormalize+
        overlapeig[i,2,j+nrank,:]=np.squeeze(np.reshape(neigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j+nrank,:,:]))/(eigvAm[j])**2+np.squeeze(np.reshape(neigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j+nrank,:,:]))/(eigvAm[j])**2# inhibitory overlap -- renormalized

        # overlap renormalizing
        # m
        overlapeig_renorm[i,0,j,:]=np.squeeze(np.reshape(meigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j,:,:]))/np.sqrt(Norm2[i,0,j,:])/np.sqrt(Norm2Am[0,j]) # excitatory overlap -- renormalized
        overlapeig_renorm[i,1,j,:]=np.squeeze(np.reshape(meigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j,:,:]))/np.sqrt(Norm2[i,1,j,:])/np.sqrt(Norm2Am[1,j]) # inhibitory overlap -- renromalized
        overlapeig_renorm[i,2,j,:]=(np.squeeze(np.reshape(meigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j,:,:]))+np.squeeze(np.reshape(meigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j,:,:])))/np.sqrt(Norm2[i,2,j,:])/np.sqrt(Norm2Am[2,j]) # overlap -- renromalized
        # n
        overlapeig_renorm[i,0,j+nrank,:]=np.squeeze(np.reshape(neigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j+nrank,:,:]))/np.sqrt(Norm2[i,0,j+nrank,:])/np.sqrt(Norm2Am[0,j+nrank]) # excitatory overlap -- renormalized
        overlapeig_renorm[i,1,j+nrank,:]=np.squeeze(np.reshape(neigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j+nrank,:,:]))/np.sqrt(Norm2[i,1,j+nrank,:])/np.sqrt(Norm2Am[1,j+nrank]) # inhibitory overlap -- renormalized
        overlapeig_renorm[i,2,j+nrank,:]=(np.squeeze(np.reshape(neigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j+nrank,:,:]))+np.squeeze(np.reshape(neigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j+nrank,:,:])))/np.sqrt(Norm2[i,2,j+nrank,:])/np.sqrt(Norm2Am[2,j+nrank])
        # statistics
        meanloadingeigclouds[i,0,j,:]=(np.mean(np.squeeze(loadingeigvec[i,0,j,:,:]),axis=0)) # m excitatory trial averaging
        meanloadingeigclouds[i,0,j+nrank,:]=(np.mean(np.squeeze(loadingeigvec[i,0,j+nrank,:,:]),axis=0)) # n excitatory trial average
        meanloadingeigclouds[i,1,j,:]=(np.mean(np.squeeze(loadingeigvec[i,1,j,:,:]),axis=0)) # m inhibitory trial average
        meanloadingeigclouds[i,1,j+nrank,:]=(np.mean(np.squeeze(loadingeigvec[i,1,j+nrank,:,:]),axis=0)) # n inhibitory trial average

        stdloadingeigclouds[i,0,j,:]=np.std(np.squeeze(loadingeigvec[i,0,j,:,:]),axis=0)
        stdloadingeigclouds[i,0,j+nrank,:]=np.std(np.squeeze(loadingeigvec[i,0,j+nrank,:,:]),axis=0)
        stdloadingeigclouds[i,1,j,:]=np.std(np.squeeze(loadingeigvec[i,1,j,:,:]),axis=0)
        stdloadingeigclouds[i,1,j+nrank,:]=np.std(np.squeeze(loadingeigvec[i,1,j+nrank,:,:]),axis=0)

        # reproduce lambda overall
        lambda_reconstruct[i,0,j,:]=np.sum(np.squeeze(meigseries[i,:,:NE,j])*np.squeeze(neigseries[i,:,:NE,j]),axis=1)
        lambda_reconstruct[i,1,j,:]=np.sum(np.squeeze(meigseries[i,:,NE:,j])*np.squeeze(neigseries[i,:,NE:,j]),axis=1)
        lambda_reconstruct[i,2,j,:]=np.sum(np.squeeze(meigseries[i,:,:,j])*np.squeeze(neigseries[i,:,:,j]),axis=1)

## Plottings for eigenvectors 
* overlap of right and left eigenvectors -- $\lambda_1,\lambda_2$
* overlap of the new and original eigenvectors -- renormalize?  
* connectivity patterns in loading space -- mean and variance of gaussian pattern

In [923]:
# FIGURE same as overlap
# MEAN
xticks = np.linspace(0.0,1.0,3)
xlims = [-0.1,1.1]
ytickm = np.linspace(0.010,0.040,3)
ylimm = [0.010,0.040]
ytickn = np.linspace(0.00,0.40,3)
ylimn = [-0.01,0.40]
fig,ax=plt.subplots(4,3,figsize=(12,8))
# FIGURE same as overlap
idxax=0
''' mean '''
mmeanE = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:,0,0,:])),axis=1)
mmeanI = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:,1,0,:])),axis=1)
''' std '''
mstdE = np.mean(np.squeeze(np.abs(stdloadingeigclouds[:,0,0,:])),axis=1)
mstdI = np.mean(np.squeeze(np.abs(stdloadingeigclouds[:,1,0,:])),axis=1)
ax[0][idxax].plot(gaverageseries,mmeanE,color='red',linewidth=1.5,alpha=1.0,label='E population')
ax[0][idxax].plot(gaverageseries,mmeanI,color='blue',linewidth=1.5,alpha=1.0,label='I population')
ax[0][idxax].plot(gaverageseries,mstdE,color='orange',linewidth=1.5,alpha=1.0,label='E population')
ax[0][idxax].plot(gaverageseries,mstdI,color='cyan',linewidth=1.5,alpha=1.0,label='I population')
# ax[0][idxax].set_xlim(xlims)
# ax[0][idxax].set_ylim(ylimm)
# ax[0][idxax].set_xticks(xticks)
# ax[0][idxax].set_yticks(ytickm)
mmeanE = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:,0,1,:])),axis=1)
mmeanI = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:,1,1,:])),axis=1)
''' std '''
mstdE = np.mean(np.squeeze(np.abs(stdloadingeigclouds[:,0,1,:])),axis=1)
mstdI = np.mean(np.squeeze(np.abs(stdloadingeigclouds[:,1,1,:])),axis=1)
ax[2][idxax].plot(gaverageseries[:],mmeanE,color='red',linewidth=1.5,alpha=1.0,label='E population')
ax[2][idxax].plot(gaverageseries[:],mmeanI,color='blue',linewidth=1.5,alpha=1.0,label='I population')
ax[2][idxax].plot(gaverageseries[:],mstdE,color='orange',linewidth=1.5,alpha=1.0,label='E population')
ax[2][idxax].plot(gaverageseries[:],mstdI,color='cyan',linewidth=1.5,alpha=1.0,label='I population')
# ax[2][idxax].set_xlim(xlims)
# ax[2][idxax].set_ylim(ylimm)
# ax[2][idxax].set_xticks(xticks)
# ax[2][idxax].set_yticks(ytickm)

nmeanE = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:,0,0+nrank,:])),axis=1)
nmeanI = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:,1,0+nrank,:])),axis=1)
''' std '''
nstdE = np.mean(np.squeeze(np.abs(stdloadingeigclouds[:,0,0+nrank,:])),axis=1)
nstdI = np.mean(np.squeeze(np.abs(stdloadingeigclouds[:,1,0+nrank,:])),axis=1)
ax[1][idxax].plot(gaverageseries,nmeanE,color='red',linewidth=1.5,alpha=1.0,label='E population')
ax[1][idxax].plot(gaverageseries,nmeanI,color='blue',linewidth=1.5,alpha=1.0,label='I population')
ax[1][idxax].plot(gaverageseries,nstdE,color='orange',linewidth=1.5,alpha=1.0,label='E population')
ax[1][idxax].plot(gaverageseries,nstdI,color='cyan',linewidth=1.5,alpha=1.0,label='I population')
ax[1][idxax].set_xlim(xlims)
ax[1][idxax].set_ylim(ylimn)
ax[1][idxax].set_xticks(xticks)
ax[1][idxax].set_yticks(ytickn)
nmeanE = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:,0,1+nrank,:])),axis=1)
nmeanI = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:,1,1+nrank,:])),axis=1)
''' std '''
nstdE = np.mean(np.squeeze(np.abs(stdloadingeigclouds[:,0,1+nrank,:])),axis=1)
nstdI = np.mean(np.squeeze(np.abs(stdloadingeigclouds[:,1,1+nrank,:])),axis=1)
ax[3][idxax].plot(gaverageseries[:],nmeanE,color='red',linewidth=1.5,alpha=1.0,label='E population')
ax[3][idxax].plot(gaverageseries[:],nmeanI,color='blue',linewidth=1.5,alpha=1.0,label='I population')
ax[3][idxax].plot(gaverageseries[:],nstdE,color='orange',linewidth=1.5,alpha=1.0,label='E population')
ax[3][idxax].plot(gaverageseries[:],nstdI,color='cyan',linewidth=1.5,alpha=1.0,label='I population')
# ax[3][idxax].set_xlim(xlims)
# ax[3][idxax].set_ylim(ylimn)
# ax[3][idxax].set_xticks(xticks)
# ax[3][idxax].set_yticks(ytickn)


In [924]:
# FIGURE same as overlap
# MEAN
xticks = np.linspace(0.0,1.0,3)
xlims = [-0.1,1.1]
ytickm = np.linspace(0.00,0.030,3)
ylimm = [0.00,0.030]
ytickn = np.linspace(0.00,0.40,3)
ylimn = [-0.01,0.40]
fig,ax=plt.subplots(4,3,figsize=(12,8))
# FIGURE same as overlap
idxxx=15
idxax=1
mmeanE = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:,0,0,:])),axis=1)
mmeanI = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:,1,0,:])),axis=1)
ax[0][idxax].plot(gaverageseries,mmeanE,color='red',linewidth=1.5,alpha=1.0,label='E population')
ax[0][idxax].plot(gaverageseries,mmeanI,color='blue',linewidth=1.5,alpha=1.0,label='I population')

mstdE = np.mean(np.squeeze(np.abs(stdloadingeigclouds[:,0,0,:])),axis=1)
mstdI = np.mean(np.squeeze(np.abs(stdloadingeigclouds[:,1,0,:])),axis=1)
ax[0][idxax].plot(gaverageseries,mstdE,color='orange',linewidth=1.5,alpha=1.0,label='E population')
ax[0][idxax].plot(gaverageseries,mstdI,color='cyan',linewidth=1.5,alpha=1.0,label='I population')
ax[0][idxax].set_xlim(xlims)
ax[0][idxax].set_ylim(ylimm)
ax[0][idxax].set_xticks(xticks)
ax[0][idxax].set_yticks(ytickm)
mmeanE = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:idxxx,0,1,:])),axis=1)
mmeanI = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:idxxx,1,1,:])),axis=1)
ax[2][idxax].plot(gaverageseries[:idxxx],mmeanE,color='red',linewidth=1.5,alpha=1.0,label='E population')
ax[2][idxax].plot(gaverageseries[:idxxx],mmeanI,color='blue',linewidth=1.5,alpha=1.0,label='I population')

mstdE = np.mean(np.squeeze(np.abs(stdloadingeigclouds[:idxxx,0,1,:])),axis=1)
mstdI = np.mean(np.squeeze(np.abs(stdloadingeigclouds[:idxxx,1,1,:])),axis=1)
ax[2][idxax].plot(gaverageseries[:idxxx],mstdE,color='orange',linewidth=1.5,alpha=1.0,label='E population')
ax[2][idxax].plot(gaverageseries[:idxxx],mstdI,color='cyan',linewidth=1.5,alpha=1.0,label='I population')
# ax[2][idxax].set_xlim(xlims)
# ax[2][idxax].set_ylim(ylimm)
# ax[2][idxax].set_xticks(xticks)
# ax[2][idxax].set_yticks(ytickm)

nmeanE = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:,0,0+nrank,:])),axis=1)
nmeanI = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:,1,0+nrank,:])),axis=1)
ax[1][idxax].plot(gaverageseries,nmeanE,color='red',linewidth=1.5,alpha=1.0,label='E population')
ax[1][idxax].plot(gaverageseries,nmeanI,color='blue',linewidth=1.5,alpha=1.0,label='I population')

nstdE = np.mean(np.squeeze(np.abs(stdloadingeigclouds[:,0,0+nrank,:])),axis=1)
nstdI = np.mean(np.squeeze(np.abs(stdloadingeigclouds[:,1,0+nrank,:])),axis=1)
ax[1][idxax].plot(gaverageseries,nstdE,color='orange',linewidth=1.5,alpha=1.0,label='E population')
ax[1][idxax].plot(gaverageseries,nstdI,color='cyan',linewidth=1.5,alpha=1.0,label='I population')
# ax[1][idxax].set_xlim(xlims)
# ax[1][idxax].set_ylim(ylimn)
# ax[1][idxax].set_xticks(xticks)
# ax[1][idxax].set_yticks(ytickn)
nmeanE = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:idxxx,0,1+nrank,:])),axis=1)
nmeanI = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:idxxx,1,1+nrank,:])),axis=1)
ax[3][idxax].plot(gaverageseries[:idxxx],nmeanE,color='red',linewidth=1.5,alpha=1.0,label='E population')
ax[3][idxax].plot(gaverageseries[:idxxx],nmeanI,color='blue',linewidth=1.5,alpha=1.0,label='I population')

nstdE = np.mean(np.squeeze(np.abs(stdloadingeigclouds[:idxxx,0,1+nrank,:])),axis=1)
nstdI = np.mean(np.squeeze(np.abs(stdloadingeigclouds[:idxxx,1,1+nrank,:])),axis=1)
ax[3][idxax].plot(gaverageseries[:idxxx],nstdE,color='orange',linewidth=1.5,alpha=1.0,label='E population')
ax[3][idxax].plot(gaverageseries[:idxxx],nstdI,color='cyan',linewidth=1.5,alpha=1.0,label='I population')
# ax[3][idxax].set_xlim(xlims)
# ax[3][idxax].set_ylim(ylimn)
# ax[3][idxax].set_xticks(xticks)
# ax[3][idxax].set_yticks(ytickn)


In [925]:
# std
xticks = np.linspace(0.0,1.0,3)
xlims = [-0.1,1.1]
ytickm = np.linspace(0.00,0.02,3)
ylimm = [-0.001,0.02]
ytickn = np.linspace(0.00,0.20,3)
ylimn = [-0.01,0.20]
# fig,ax=plt.subplots(2,3,figsize=(12,8))
# FIGURE same as overlap
for j in range(nrank):
    mstdE = np.mean(np.squeeze(np.abs(stdloadingeigclouds[:,0,j,:])),axis=1)
    mstdI = np.mean(np.squeeze(np.abs(stdloadingeigclouds[:,1,j,:])),axis=1)
    ax[0][idxax].plot(gaverageseries,mstdE,color='red',linewidth=1.5,alpha=1.0,label='E population')
    ax[0][idxax].plot(gaverageseries,mstdI,color='blue',linewidth=1.5,alpha=1.0,label='I population')
    ax[0][idxax].set_xlim(xlims)
    ax[0][idxax].set_ylim(ylimm)
    ax[0][idxax].set_xticks(xticks)
    ax[0][idxax].set_yticks(ytickm)

    nstdE = np.mean(np.squeeze(np.abs(stdloadingeigclouds[:,0,j+nrank,:])),axis=1)
    nstdI = np.mean(np.squeeze(np.abs(stdloadingeigclouds[:,1,j+nrank,:])),axis=1)
    ax[1][idxax].plot(gaverageseries,nstdE,color='red',linewidth=1.5,alpha=1.0,label='E population')
    ax[1][idxax].plot(gaverageseries,nstdI,color='blue',linewidth=1.5,alpha=1.0,label='I population')
    # ax[1][idxax].set_xlim(xlims)
    # ax[1][idxax].set_ylim(ylimn)
    # ax[1][idxax].set_xticks(xticks)
    # ax[1][idxax].set_yticks(ytickn)

In [926]:
xticks = np.linspace(0.0,1.0,3)
xlims = [-0.1,1.1]
yticklambda = np.linspace(-15,15,3)
ylimlambda = [-15.1,15.1]
# ytickpc = np.linspace(0.75,1.0,2) # homogeneous
# ytickpc = np.linspace(0.70,1.0,2) # RII
ytickpc = np.linspace(0.50,1.0,2) # new RII
# ylimpc = [0.74,1.01] # homogeneous
# ylimpc = [0.69,1.01] # RII
ylimpc = [0.49,1.01] # new RII
fig,ax=plt.subplots(2,3,figsize=(12,4))
# FIGURE 1
for i in range(ngavg):
    for j in range(nrank):
        # RECONSTRUCT EIGENVALUES
        # m1
        ax[j][0].scatter(gaverageseries[i]*np.ones(nbatch),lambda_reconstruct[i,0,j,:],s=2,color='orange',alpha=0.5)
        ax[j][0].scatter(gaverageseries[i]*np.ones(nbatch),lambda_reconstruct[i,1,j,:],s=2,color='cyan',alpha=0.5)
        ax[j][0].scatter(gaverageseries[i]*np.ones(nbatch),lambda_reconstruct[i,2,j,:],s=2,color='gray',alpha=0.5)

        
        # OVERLAP
        # m1
        # ax[j][1].scatter(gaverageseries[i]*np.ones(nbatch),overlapeig[i,0,j,:],s=5,color='red',alpha=0.5)
        # ax[j][1].scatter(gaverageseries[i]*np.ones(nbatch),overlapeig[i,1,j,:],s=5,color='blue',alpha=0.5)
        ax[j][1].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(overlapeig_renorm[i,0,j,:]),s=2,color='orange',alpha=0.5)
        ax[j][1].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(overlapeig_renorm[i,1,j,:]),s=2,color='cyan',alpha=0.5)
        ax[j][1].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(overlapeig_renorm[i,2,j,:]),s=2,color='gray',alpha=0.5)


        # n1
        ax[j][2].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(overlapeig_renorm[i,0,j+nrank,:]),s=2,color='orange',alpha=0.5)
        ax[j][2].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(overlapeig_renorm[i,1,j+nrank,:]),s=2,color='cyan',alpha=0.5)
        ax[j][2].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(overlapeig_renorm[i,2,j+nrank,:]),s=2,color='gray',alpha=0.5)

for j in range(nrank):
    if j==0:
        iend = 20
    else:
        iend=20
    lambdaE = np.mean(np.squeeze(lambda_reconstruct[:,0,j,:]),axis=1)
    lambdaI = np.mean(np.squeeze(lambda_reconstruct[:,1,j,:]),axis=1)
    lambdaT = np.mean(np.squeeze(lambda_reconstruct[:,2,j,:]),axis=1)
    ax[j][0].plot(gaverageseries[:iend],lambdaE[:iend],color='red',linewidth=1.5,alpha=1.0,label='E population')
    ax[j][0].plot(gaverageseries[:iend],lambdaI[:iend],color='blue',linewidth=1.5,alpha=1.0,label='E population')
    ax[j][0].plot(gaverageseries[:iend],lambdaT[:iend],color='black',linewidth=1.5,alpha=1.0,label='E population')
    ax[j][0].set_xlim(xlims)
    ax[j][0].set_ylim(ylimlambda)
    ax[j][0].set_xticks(xticks)
    ax[j][0].set_yticks(yticklambda)

    mE = np.mean(np.squeeze(np.abs(overlapeig_renorm[:,0,j,:])),axis=1)
    mI = np.mean(np.squeeze(np.abs(overlapeig_renorm[:,1,j,:])),axis=1)
    mT = np.mean(np.squeeze(np.abs(overlapeig_renorm[:,2,j,:])),axis=1)
    ax[j][1].plot(gaverageseries[:iend],mE[:iend],color='red',linewidth=1.5,alpha=1.0,label='E population')
    ax[j][1].plot(gaverageseries[:iend],mI[:iend],color='blue',linewidth=1.5,alpha=1.0,label='I population')
    ax[j][1].plot(gaverageseries[:iend],mT[:iend],color='black',linewidth=1.5,alpha=1.0,label='Total')
    ax[j][1].set_xlim(xlims)
    ax[j][1].set_ylim(ylimpc)
    ax[j][1].set_xticks(xticks)
    ax[j][1].set_yticks(ytickpc)

    nE = np.mean(np.squeeze(np.abs(overlapeig_renorm[:,0,j+nrank,:])),axis=1)
    nI = np.mean(np.squeeze(np.abs(overlapeig_renorm[:,1,j+nrank,:])),axis=1)
    nT = np.mean(np.squeeze(np.abs(overlapeig_renorm[:,2,j+nrank,:])),axis=1)
    ax[j][2].plot(gaverageseries[:iend],nE[:iend],color='red',linewidth=1.5,alpha=1.0,label='E population')
    ax[j][2].plot(gaverageseries[:iend],nI[:iend],color='blue',linewidth=1.5,alpha=1.0,label='I population')
    ax[j][2].plot(gaverageseries[:iend],nT[:iend],color='black',linewidth=1.5,alpha=1.0,label='Total')
    ax[j][2].set_xlim(xlims)
    ax[j][2].set_ylim(ylimpc)
    ax[j][2].set_xticks(xticks)
    ax[j][2].set_yticks(ytickpc)
ax[0][2].legend()

In [927]:
# ytickavg = np.linspace(0.022,0.032,3)
# ylimavg = [0.0219,0.0321]
# new R2
ytickavg = np.linspace(0.002,0.032,3)
ylimavg = [0.0019,0.0321]
fig,ax=plt.subplots(2,2,figsize=(12,4))
# FIGURE same as overlap
for i in range(ngavg):
    for j in range(nrank):
        # mean m1
        ax[j][0].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,0,j,:])/np.sqrt(Norm2[i,0,j,:]),s=5,color='orange',alpha=0.25)
        ax[j][0].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,1,j,:])/np.sqrt(Norm2[i,1,j,:]),s=5,color='cyan',alpha=0.25)
        # mean n1
        ax[j][1].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,0,nrank+j,:])/np.sqrt(Norm2[i,0,j+nrank,:]),s=5,color='orange',alpha=0.25)
        ax[j][1].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,1,nrank+j,:])/np.sqrt(Norm2[i,1,j+nrank,:]),s=5,color='cyan',alpha=0.25)

for j in range(nrank):
    mmeanE = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:,0,j,:]))/np.squeeze(np.sqrt(Norm2[:,0,j,:])),axis=1)
    mmeanI = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:,1,j,:]))/np.squeeze(np.sqrt(Norm2[:,1,j,:])),axis=1)
    ax[j][0].plot(gaverageseries,mmeanE,color='red',linewidth=1.5,alpha=1.0,label='E population')
    ax[j][0].plot(gaverageseries,mmeanI,color='blue',linewidth=1.5,alpha=1.0,label='I population')
    ax[j][0].set_xlim(xlims)
    ax[j][0].set_ylim(ylimavg)
    ax[j][0].set_xticks(xticks)
    ax[j][0].set_yticks(ytickavg)

    nmeanE = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:,0,j+nrank,:]))/np.squeeze(np.sqrt(Norm2[:,0,j+nrank,:])),axis=1)
    nmeanI = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:,1,j+nrank,:]))/np.squeeze(np.sqrt(Norm2[:,1,j+nrank,:])),axis=1)
    ax[j][1].plot(gaverageseries,nmeanE,color='red',linewidth=1.5,alpha=1.0,label='E population')
    ax[j][1].plot(gaverageseries,nmeanI,color='blue',linewidth=1.5,alpha=1.0,label='I population')
    ax[j][1].set_xlim(xlims)
    ax[j][1].set_ylim(ylimavg)
    ax[j][1].set_xticks(xticks)
    ax[j][1].set_yticks(ytickavg)
ax[0][1].legend()
fig,ax=plt.subplots(2,2,figsize=(12,4))
# FIGURE absolute value
for i in range(ngavg):
    for j in range(nrank):
        # mean m1
        ax[j][0].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,0,j,:]),s=5,color='red',alpha=0.5)
        ax[j][0].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,1,j,:]),s=5,color='blue',alpha=0.5)
        # mean n1
        ax[j][1].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,0,nrank+j,:]),s=5,color='red',alpha=0.5)
        ax[j][1].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,1,nrank+j,:]),s=5,color='blue',alpha=0.5)

## quantitative measurements

In [928]:
# statistics
mAm2,nAm2=np.zeros((2,2)),np.zeros((2,2))
for i in range(nrank):
    mAm2[:,i]=meigvecAm[1::NE,i]
    nAm2[:,i]=neigvecAm[1::NE,i]
invmbar,invnbar=la.inv(mAm2),la.inv(nAm2)
meanloadingeig_trialsavg=np.zeros((ngavg,2,nrank*2))
# ''' a=b change '''
# # m1--
# meanloadingeig_trialsavg[:,:,0] = -np.mean(np.abs(np.squeeze(meanloadingeigclouds[:,:,0,:])),axis=2)#np.squeeze(meanloadingeigclouds[:,:,:,12])#
# meanloadingeig_trialsavg[:,:,1] = -np.mean(np.abs(np.squeeze(meanloadingeigclouds[:,:,1,:])),axis=2)
# meanloadingeig_trialsavg[:,:,2] = np.mean(np.abs(np.squeeze(meanloadingeigclouds[:,:,2,:])),axis=2)
# meanloadingeig_trialsavg[:,0,2]*=-1
# meanloadingeig_trialsavg[:,:,3] = np.mean(np.abs(np.squeeze(meanloadingeigclouds[:,:,3,:])),axis=2)
# meanloadingeig_trialsavg[:,1,3]*=-1
''' a!=b '''
# m1--
# meanloadingeig_trialsavg[:,:,0] = -np.mean(np.abs(np.squeeze(meanloadingeigclouds[:,:,0,:])),axis=2)#np.squeeze(meanloadingeigclouds[:,:,:,12])#
# meanloadingeig_trialsavg[:,:,1] = -np.mean(np.abs(np.squeeze(meanloadingeigclouds[:,:,1,:])),axis=2)
# meanloadingeig_trialsavg[:,:,2] = np.mean(np.abs(np.squeeze(meanloadingeigclouds[:,:,2,:])),axis=2)
# meanloadingeig_trialsavg[:,0,2]*=-1
# meanloadingeig_trialsavg[:,:,3] = np.mean(np.abs(np.squeeze(meanloadingeigclouds[:,:,3,:])),axis=2)
# meanloadingeig_trialsavg[:,1,3]*=-1

meanloadingeig_trialsavg[:,:,0] = -np.mean(np.abs(np.squeeze(meanloadingeigclouds[:,:,0,:])),axis=2)#np.squeeze(meanloadingeigclouds[:,:,:,12])#
meanloadingeig_trialsavg[:,:,1] = np.mean(np.abs(np.squeeze(meanloadingeigclouds[:,:,1,:])),axis=2)
meanloadingeig_trialsavg[:,:,2] = np.mean(np.abs(np.squeeze(meanloadingeigclouds[:,:,2,:])),axis=2)
meanloadingeig_trialsavg[:,0,2]*=-1
meanloadingeig_trialsavg[:,:,3] = np.mean(np.abs(np.squeeze(meanloadingeigclouds[:,:,3,:])),axis=2)
meanloadingeig_trialsavg[:,0,3]*=-1
Oeigseries=np.zeros((ngavg,2,nrank*2))
for i in range(ngavg):
    m_mean = np.squeeze(meanloadingeig_trialsavg[i,:,:nrank])
    n_mean = np.squeeze(meanloadingeig_trialsavg[i,:,nrank:])
    Oeigm,Oeign = invmbar@m_mean,invnbar@n_mean
    Oeigseries[i,:,:nrank],Oeigseries[i,:,nrank:]=Oeigm.copy(),Oeign.copy()

# three copy
idx = 9
grand = gaverageseries[idx]*np.ones(4)*np.sqrt(2.0/3.0)
grand[0]=0
xee,xei,xie,xii=grand[0],grand[1],grand[2],grand[3]
gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
eigvgm,eigvecgm=la.eig(gmat)
# print(eigvgm)
print(Oeigseries[idx,:,:nrank])
print(Oeigseries[idx,:,nrank:])
attt=(1-gaverageseries[idx]**2/eigvAm[:nrank]**2)#eigvgm**2/eigvAm[:nrank]**2)
print(np.sqrt(attt))
## plot
fig,ax=plt.subplots(1,2,figsize=(12,5))
''' a=b ''' 
# yalims=[0.89,1.11]
# yaticks=np.linspace(0.9,1.1,3)
''' a!=b ''' 
yalims=[0.49,1.51]
yaticks=np.linspace(0.5,1.5,3)
for i in range(nrank):
    if i==0:
        iend = 20
    else:
        iend=20
    # FIGURE absolute value
    ax[i].plot(gaverageseries[:iend],Oeigseries[:iend,i,i],color='red',linewidth=1.5,label=r'$\alpha_{m}^E$')
    ax[i].plot(gaverageseries[:iend],Oeigseries[:iend,i,i+nrank],color='blue',linewidth=1.5,label=r'$\alpha_{n}^I$')
    ax[i].plot(gaverageseries,np.sqrt(1-gaverageseries**2/eigvAm[i]**2),color='black',linestyle='--',linewidth=1.5,label=r'$\alpha_m^{theo}$')
    ax[i].plot(gaverageseries,1/np.sqrt(1-gaverageseries**2/eigvAm[i]**2),color='gray',linestyle='--',linewidth=1.5,label=r'$\alpha_n^{theo}$')
    ax[i].set_xlim(xlims)
    ax[i].set_xticks(xticks)
    ax[i].set_ylim(yalims)
    ax[i].set_yticks(yaticks)
# ax[1].legend()


[[9.96470364e-01 9.54258472e-04]
 [2.71011811e-03 9.32467920e-01]]
[[ 1.00471985e+00 -7.97712945e-04]
 [-3.28235789e-03  1.06799455e+00]]
[0.99476232+0.j 0.92409619+0.j]


In [433]:
Oeigseries[:,1,]

array([-1.00025096, -1.00134822, -1.00213784, -1.00730093, -1.00985742,
       -1.01564095, -1.02462289, -1.03230674, -1.05014872, -1.0595593 ,
       -1.0829692 , -1.10449351, -1.14307025, -1.20408384, -1.26507768,
       -1.40173683, -1.05138942, -0.57342441, -0.33823233, -0.1994065 ])

In [507]:
for i in range(20):
    mEnum, mInum=Oeigseries[i,:,:nrank],Oeigseries[i,:,nrank:]
    print(mEnum,mInum.T)

[[9.99953518e-01 3.30112157e-05]
 [7.17474652e-06 9.99292885e-01]] [[ 1.00005542e+00 -1.27622254e-04]
 [-2.09238680e-06  1.00070003e+00]]
[[ 9.99609666e-01  4.25564829e-05]
 [-1.97958111e-04  9.97108677e-01]] [[ 1.00023660e+00 -3.71902765e-05]
 [ 4.97171086e-05  1.00285924e+00]]
[[ 9.99326592e-01 -1.73736632e-04]
 [-2.20565602e-04  9.93288173e-01]] [[1.00034237e+00 7.66298810e-04]
 [5.77626182e-05 1.00706566e+00]]
[[9.99671732e-01 7.04856636e-04]
 [5.57780655e-04 9.88930559e-01]] [[ 1.00095059e+00 -2.68065874e-03]
 [-1.47315789e-04  1.00995929e+00]]
[[9.99201022e-01 4.24470008e-04]
 [5.44684913e-04 9.82074418e-01]] [[ 1.00108151e+00 -1.78016481e-03]
 [-1.17769637e-04  1.01638243e+00]]
[[ 9.97606594e-01 -9.50043391e-04]
 [-5.55931038e-04  9.72581337e-01]] [[1.00171491e+00 4.03644192e-03]
 [1.70600243e-04 1.03064327e+00]]
[[ 9.97133061e-01 -1.17240914e-04]
 [-2.96570683e-04  9.63281269e-01]] [[ 1.00317497e+00 -6.47542086e-05]
 [ 2.00412554e-04  1.03927290e+00]]
[[0.99847159 0.00189295]
 

In [565]:
lims=[-0.81,0.81]
ticks=np.linspace(-0.8,0.8,3)
fig,ax=plt.subplots(figsize=(4,4))
IDXGAVG,IDXRANK=8,12
## scatter and direction
nE1,nI1=np.squeeze(loadingeigvec[IDXGAVG,0,2,:NE,IDXRANK]),np.squeeze(loadingeigvec[IDXGAVG,1,2,:,IDXRANK])
nE2,nI2=np.squeeze(loadingeigvec[IDXGAVG,0,3,:NE,IDXRANK]),np.squeeze(loadingeigvec[IDXGAVG,1,3,:,IDXRANK])
meanE1,meanE2=np.mean(nE1),np.mean(nE2)
meanI1,meanI2=np.mean(nI1),np.mean(nI2)
# fig,ax=plt.subplots(figsize=(4,4))
# FIGURE same as overlap
ax.scatter(nE1,nE2,s=5,color='red',alpha=0.5)
ax.scatter(nI1,nI2,s=5,color='blue',alpha=0.5)
for i in range(2):
    nAm2[:,i]=nAm2[:,i]/eigvAm[i]
covnAm=nAm2.T@nAm2
_,eigveccov=la.eig(covnAm)
ax.plot([meanE1,meanE1+eigveccov[0,0]/5],[meanE2,meanE2+eigveccov[1,0]/5],linestyle='--',color='pink',linewidth=2.0)
ax.plot([meanE1,meanE1+eigveccov[0,1]/5],[meanE2,meanE2+eigveccov[1,1]/5],linestyle='--',color='pink',linewidth=2.0)
ax.plot([meanI1,meanI1+eigveccov[0,0]/5],[meanI2,meanI2+eigveccov[1,0]/5],linestyle='--',color='cyan',linewidth=2.0)
ax.plot([meanI1,meanI1+eigveccov[0,1]/5],[meanI2,meanI2+eigveccov[1,1]/5],linestyle='--',color='cyan',linewidth=2.0)
ax.set_xlim(lims)
ax.set_xticks(ticks)
ax.set_ylim(lims)
ax.set_yticks(ticks)